#Bio BERT

## Trying new logic Experminet 1

In [ ]:
import pandas as pd

# Read both CSV files
df_abstract = pd.read_csv('/content/drive/.shortcut-targets-by-id/1lhuR_Wtv2FHzNlS9dEdH08FjrmhgsiBy/Dataset/bioscope_abstract.csv')
df_full = pd.read_csv('/content/drive/.shortcut-targets-by-id/1lhuR_Wtv2FHzNlS9dEdH08FjrmhgsiBy/Dataset/bioscope_full.csv')

# df = pd.concat([df_abstract, df_full], ignore_index=True)
df = df_full
# Print the combined DataFrame
print(len(df))


2469


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [ ]:

for idx, row in df.iterrows():
    # Parse cue and scope spans, handling NaN values
    cue_span = eval(row['cue_span']) if pd.notna(row['cue_span']) and row['cue_span'] != 'nan' else []
    scope_span = eval(row['scope_span']) if pd.notna(row['scope_span']) and row['scope_span'] != 'nan' else []

    # Tokenize the sentence without adding special tokens
    tokens = tokenizer(row['sentence'], padding="max_length", truncation=True, max_length=64, return_tensors="pt", return_offsets_mapping=True, add_special_tokens=False)
    offsets = tokens['offset_mapping'][0].tolist()

    # Initialize BIO tags for cue, scope, and combined tags
    cue_bio_tags = ['O'] * len(tokens['input_ids'][0])
    scope_bio_tags = ['O'] * len(tokens['input_ids'][0])
    combined_bio_tags = ['O'] * len(tokens['input_ids'][0])

    # Define start and end for cue and scope spans
    cue_start, cue_end = cue_span[0] if cue_span else (None, None)
    scope_start, scope_end = scope_span if scope_span else (None, None)

    # Filter tokens that lie within cue and scope spans, excluding special tokens
    cue_tokens = [
        i for i, (start, end) in enumerate(offsets)
        if start >= cue_start and end <= cue_end and tokens['input_ids'][0][i].item() not in tokenizer.all_special_ids
    ] if cue_start is not None else []

    scope_tokens = [
        i for i, (start, end) in enumerate(offsets)
        if start >= scope_start and end <= scope_end and tokens['input_ids'][0][i].item() not in tokenizer.all_special_ids
    ] if scope_start is not None else []

    # Modify scope tokens to exclude any tokens that overlap with cue tokens
    modified_scope_tokens = [
        token for token in scope_tokens if not any(start >= cue_start and end <= cue_end for start, end in [offsets[token]])
    ]

    # Assign BIO tags for cue tokens
    for i in cue_tokens:
        if i == cue_tokens[0]:
            cue_bio_tags[i] = 'B-CUE'
        else:
            cue_bio_tags[i] = 'I-CUE'

    # Assign BIO tags for modified scope tokens, ensuring no overlap with cue tags
    for i in modified_scope_tokens:
        if cue_bio_tags[i] == 'O':
            if i == modified_scope_tokens[0]:
                scope_bio_tags[i] = 'B-SCOPE'
            else:
                scope_bio_tags[i] = 'I-SCOPE'

    # Combine cue and scope tags into one set of combined tags
    for i in range(len(cue_bio_tags)):
        if cue_bio_tags[i] != 'O':
            combined_bio_tags[i] = cue_bio_tags[i]
        elif scope_bio_tags[i] != 'O':
            combined_bio_tags[i] = scope_bio_tags[i]

    # Convert tokens and offsets for display in the DataFrame
    tokenized_words = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
    token_spans = [f"{start}-{end}" for start, end in offsets]
    cue_tokens_str = tokenizer.convert_ids_to_tokens([tokens['input_ids'][0][i] for i in cue_tokens])
    scope_tokens_str = tokenizer.convert_ids_to_tokens([tokens['input_ids'][0][i] for i in modified_scope_tokens])

    # Update DataFrame with the calculated values
    df.at[idx, 'Tokenized_Tokens'] = ' '.join(tokenized_words)
    df.at[idx, 'Token_Spans'] = ' '.join(token_spans)
    df.at[idx, 'Cue_BIO_Tags'] = ' '.join(cue_bio_tags)
    df.at[idx, 'Scope_BIO_Tags'] = ' '.join(scope_bio_tags)
    df.at[idx, 'Combined_BIO_Tags'] = ' '.join(combined_bio_tags)
    df.at[idx, 'Cue_Tokens'] = ' '.join(cue_tokens_str)
    df.at[idx, 'Scope_Tokens'] = ' '.join(scope_tokens_str)
    df.at[idx, 'Modified_Cue_Span'] = str(cue_tokens)
    df.at[idx, 'Modified_Scope_Span'] = str(modified_scope_tokens)

print("Done")



Done


code to stroe the converted tags

In [ ]:
import json

def count_labels_per_negated_row(df, target_column, label_column, output_file):
    negated_rows_counts = []

    for _, row in df.iterrows():
        if row[target_column] == 1:

            label_counts = {}
            labels = row[label_column].split()
            for label in labels:
                label_counts[label] = label_counts.get(label, 0) + 1


            negated_rows_counts.append({
                "row_index": int(_),
                "labels": label_counts,
                "sentence": row['sentence'],
                "cue_span": row['cue_span'],
                "cue_tokens": row['Cue_Tokens'],
                "scope_span": row['scope_span'],
                "scope_tokens": row['Scope_Tokens'],
                "modified_cue_span": row['Modified_Cue_Span'],
                "modified_scope_span": row['Modified_Scope_Span']
            })

    with open(output_file, 'w') as f:
        json.dump(negated_rows_counts, f, indent=4)

    print(f"Results saved to {output_file}")

output_file = "/content/drive/MyDrive/NLP-ProjectFinal/BioBERT/negated_rows_labels.json"
count_labels_per_negated_row(df, target_column='target', label_column='Combined_BIO_Tags', output_file=output_file)


Results saved to /content/drive/MyDrive/NLP-ProjectFinal/BioBERT/negated_rows_labels.json


In [ ]:
def count_negated_non_negated(df, target_column):
    counts = {0: 0, 1: 0}

    for _, row in df.iterrows():
        counts[row[target_column]] += 1

    return counts

counts = count_negated_non_negated(df, target_column='target')

print("Counts:")
print(f"Non-negated (0): {counts[0]}")
print(f"Negated (1): {counts[1]}")


Counts:
Non-negated (0): 12368
Negated (1): 2094


In [ ]:
import json
with open('dffull.json', 'w') as json_file:
    json.dump(df.to_dict(orient='records'), json_file, indent=2)

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

def preprocess_data(example):
    encoding = tokenizer(
        example['Tokenized_Tokens'].split(),
        padding="max_length",
        truncation=True,
        max_length=64,
        return_tensors="pt",
        add_special_tokens=False,
        is_split_into_words=True
    )

    # Aligning BIO labels with the tokens
    labels = example['Combined_BIO_Tags'].split()
    label_map = {'O': 0, 'B-CUE': 1, 'I-CUE': 2, 'B-SCOPE': 3, 'I-SCOPE': 4}
    encoding['labels'] = [label_map.get(label, 0) for label in labels]

    return encoding


processed_data = []
for index, row in df.iterrows():
    encoding = preprocess_data(row)
    processed_data.append(encoding)


train_dataset = Dataset.from_dict({
    'input_ids': [item['input_ids'].squeeze().tolist() for item in processed_data],
    'attention_mask': [item['attention_mask'].squeeze().tolist() for item in processed_data],
    'labels': [item['labels'] for item in processed_data]
})


train_test_split = train_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

def make_model_contiguous(model):
    for param in model.parameters():
        if not param.is_contiguous():
            param.data = param.data.contiguous()

# Model setup
model = AutoModelForTokenClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.1", num_labels=5)  # 5 labels: O, B-CUE, I-CUE, B-SCOPE, I-SCOPE
make_model_contiguous(model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move model to GPU

# %raining arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    learning_rate=5e-5,
    report_to="none",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

trainer.train()

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-5-a8951cb8623c>:86: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.687600,0.408446
2,0.129900,0.128146
3,0.132600,0.117909
4,0.106700,0.104014
5,0.075900,0.083033
6,0.066900,0.080903
7,0.067400,0.077458
8,0.070400,0.083211
9,0.053100,0.073203
10,0.056800,0.084406


TrainOutput(global_step=465, training_loss=0.13953147886260864, metrics={'train_runtime': 332.8329, 'train_samples_per_second': 89.009, 'train_steps_per_second': 1.397, 'total_flos': 967640796720000.0, 'train_loss': 0.13953147886260864, 'epoch': 15.0})

In [ ]:
eval_dataset[23]
tokenized_words = tokenizer.convert_ids_to_tokens(eval_dataset[23]['input_ids'])
word = tokenizer.convert_ids_to_tokens(eval_dataset[23]['labels'])
# token_spans = [f"{start}-{end}" for start, end in eval_dataset[416]['offset_mapping']]
print(tokenized_words)
# print(word)

['this', 'recognition', 'does', 'not', 'require', 'any', 'of', 'the', 'known', 'r', '#', '#', 'o', '##x', 'r', '#', '#', 'na', '##s', '.', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
['[PAD]', '[PAD]', '[PAD]', '[unused1]', '[unused3]', '[unused4]', '[unused4]', '[unused4]', '[unused4]', '[unused4]', '[unused4]', '[unused4]', '[unused4]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

 {
    "test_id": 23,
    "Text": This recognition does not require any of the known roX RNAs.
    "true_labels": [
      "O",
      "O",
      "O",
      "B-CUE",
      "B-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O"
    ],
    "predicted_labels": [
      "O",
      "O",
      "O",
      "B-CUE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O"
    ]
  },

In [ ]:
eval_dataset[483]
tokenized_words = tokenizer.convert_ids_to_tokens(eval_dataset[483]['input_ids'])
# word = tokenizer.convert_ids_to_tokens(eval_dataset[23]['labels'])
# token_spans = [f"{start}-{end}" for start, end in eval_dataset[416]['offset_mapping']]
print(tokenized_words[25:])
# print(word)

[',', 'with', 'no', 'clear', 'superiority', 'having', 'been', 'established', 'between', 'the', 'two', '.', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


Example of model behaviour for model hallucanating from 25 token and treating it as I-scope in labels

In the machine learning field, ensemble methods have coexisted for many years with non-ensemble methods, with no clear superiority having been established between the two.

  {
    "test_id": 483,
    "true_labels": [
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O"
    ],
    "predicted_labels": [
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "I-SCOPE",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O",
      "O"
    ]
  },

### **Token-level Classification Metrics**

In [ ]:
import json
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Function to compute metrics, including per-label metrics
def compute_metrics(predictions, true_labels, label_map):
    # Flatten the predictions and labels to be per token
    predicted_labels = predictions.argmax(axis=-1)
    predicted_labels = predicted_labels.flatten()
    true_labels = true_labels.flatten()

    # Mask to ignore padding (where label == -100)
    mask = true_labels != -100
    predicted_labels = predicted_labels[mask]
    true_labels = true_labels[mask]

    metrics = {
        'accuracy': accuracy_score(true_labels, predicted_labels),
        'precision_macro': precision_score(true_labels, predicted_labels, average='macro'),
        'recall_macro': recall_score(true_labels, predicted_labels, average='macro'),
        'f1_macro': f1_score(true_labels, predicted_labels, average='macro'),
        'precision_micro': precision_score(true_labels, predicted_labels, average='micro'),
        'recall_micro': recall_score(true_labels, predicted_labels, average='micro'),
        'f1_micro': f1_score(true_labels, predicted_labels, average='micro'),
        'precision_weighted': precision_score(true_labels, predicted_labels, average='weighted'),
        'recall_weighted': recall_score(true_labels, predicted_labels, average='weighted'),
        'f1_weighted': f1_score(true_labels, predicted_labels, average='weighted')
    }

    # Calculate metrics for each label separately
    unique_labels = np.unique(true_labels)
    label_metrics = {}
    for label in unique_labels:
        label_name = label_map.get(label, f'Label_{label}')
        label_metrics[label_name] = {
            'precision': precision_score(true_labels, predicted_labels, labels=[label], average=None)[0],
            'recall': recall_score(true_labels, predicted_labels, labels=[label], average=None)[0],
            'f1': f1_score(true_labels, predicted_labels, labels=[label], average=None)[0]
        }

    # Add per-label metrics to the main metrics dictionary
    metrics['per_label'] = label_metrics

    return metrics

# label map
label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


predictions, label_ids, metrics = trainer.predict(eval_dataset)
predicted_labels = predictions.argmax(axis=-1)
reverse_label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}

results_dict = []


for i in range(len(eval_dataset)):

    true_labels = label_ids[i]
    pred_labels = predicted_labels[i]

    # Convert true labels and predicted labels back to BIO tags
    true_labels_str = [reverse_label_map.get(label, 'O') for label in true_labels]
    pred_labels_str = [reverse_label_map.get(label, 'O') for label in pred_labels]

    # Add to dictionary
    results_dict.append({
        'test_id': i,
        'true_labels': true_labels_str,
        'predicted_labels': pred_labels_str,
    })

# Save the predictions for each sample to a JSON file
with open('TLC_predictions_results.json', 'w') as json_file:
    json.dump(results_dict, json_file, indent=2)

# Calculate detailed metrics on the entire test set
detailed_metrics = compute_metrics(predictions, label_ids, label_map)

# Save detailed metrics to JSON for later comparison
with open('TLC_detailed_metrics.json', 'w') as json_file:
    json.dump(detailed_metrics, json_file, indent=2)

# Print overall metrics and per-label metrics
print("\nOverall Metrics:")
for metric_name, metric_value in detailed_metrics.items():
    if metric_name != 'per_label':  # Exclude per-label metrics here
        print(f"{metric_name}: {metric_value:.4f}")

print("\nPer-Label Metrics:")
for label, scores in detailed_metrics['per_label'].items():
    print(f"\nMetrics for {label}:")
    for metric_name, score in scores.items():
        print(f"{metric_name}: {score:.4f}")


Evaluation results: {'eval_loss': 0.07906699925661087, 'eval_runtime': 1.8476, 'eval_samples_per_second': 267.379, 'eval_steps_per_second': 4.33, 'epoch': 15.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Overall Metrics:
accuracy: 0.9762
precision_macro: 0.4525
recall_macro: 0.3067
f1_macro: 0.3150
precision_micro: 0.9762
recall_micro: 0.9762
f1_micro: 0.9762
precision_weighted: 0.9739
recall_weighted: 0.9762
f1_weighted: 0.9743

Per-Label Metrics:

Metrics for O:
precision: 0.9864
recall: 0.9904
f1: 0.9884

Metrics for B-CUE:
precision: 0.8000
recall: 0.0563
f1: 0.1053

Metrics for I-CUE:
precision: 0.0000
recall: 0.0000
f1: 0.0000

Metrics for B-SCOPE:
precision: 0.0000
recall: 0.0000
f1: 0.0000

Metrics for I-SCOPE:
precision: 0.4761
recall: 0.4870
f1: 0.4815


### **Span Matching Metrics**

In [ ]:
import numpy as np

# Function to extract spans
def extract_spans(labels, label_map):
    spans = []
    start, end, label = None, None, None

    for i, label_id in enumerate(labels):
        if label_id != 0:  # Not 'O' label
            try:
                current_label = label_map[label_id].split('-')[-1]  # Get entity type
            except KeyError:
                print(f"KeyError: label_id {label_id} not found in label_map")
                continue  # Skip if the label_id is not in label_map
            if start is None:  # Start of a new span
                start = i
                label = current_label
            elif current_label != label or label_id == 1:  # New entity or a B- tag
                end = i - 1
                spans.append((start, end, label))
                start, label = i, current_label
        else:
            if start is not None:
                end = i - 1
                spans.append((start, end, label))
                start, label = None, None
    if start is not None:  # End span if it reaches end of sequence
        spans.append((start, len(labels) - 1, label))

    return spans

def calculate_overlap_and_strict_metrics(predictions, label_ids, label_map):
    # Initialize counts for strict and overlap matching
    strict_tp, strict_fp, strict_fn = 0, 0, 0
    overlap_tp, overlap_fp, overlap_fn = 0, 0, 0
    for i in range(len(predictions)):
        true_labels = label_ids[i]
        pred_labels = predictions[i]
        true_spans = extract_spans(true_labels, label_map)
        pred_spans = extract_spans(pred_labels, label_map)
        for pred_span in pred_spans:
            if pred_span in true_spans:
                strict_tp += 1
            else:
                strict_fp += 1
        for true_span in true_spans:
            if true_span not in pred_spans:
                strict_fn += 1

        # Overlap matching: Count partial matches as true positives
        for pred_span in pred_spans:
            overlap_found = False
            for true_span in true_spans:
                if (
                    pred_span[2] == true_span[2]  # Same label
                    and (pred_span[0] <= true_span[1] and pred_span[1] >= true_span[0])  # Overlap
                ):
                    overlap_tp += 1
                    overlap_found = True
                    break
            if not overlap_found:
                overlap_fp += 1
        for true_span in true_spans:
            overlap_found = False
            for pred_span in pred_spans:
                if (
                    pred_span[2] == true_span[2]  # Same label
                    and (pred_span[0] <= true_span[1] and pred_span[1] >= true_span[0])  # Overlap
                ):
                    overlap_found = True
                    break
            if not overlap_found:
                overlap_fn += 1

    strict_precision = strict_tp / (strict_tp + strict_fp) if strict_tp + strict_fp > 0 else 0.0
    strict_recall = strict_tp / (strict_tp + strict_fn) if strict_tp + strict_fn > 0 else 0.0
    strict_f1 = 2 * (strict_precision * strict_recall) / (strict_precision + strict_recall) if strict_precision + strict_recall > 0 else 0.0

    overlap_precision = overlap_tp / (overlap_tp + overlap_fp) if overlap_tp + overlap_fp > 0 else 0.0
    overlap_recall = overlap_tp / (overlap_tp + overlap_fn) if overlap_tp + overlap_fn > 0 else 0.0
    overlap_f1 = 2 * (overlap_precision * overlap_recall) / (overlap_precision + overlap_recall) if overlap_precision + overlap_recall > 0 else 0.0

    return {
        "strict": {"precision": strict_precision, "recall": strict_recall, "f1": strict_f1},
        "overlap": {"precision": overlap_precision, "recall": overlap_recall, "f1": overlap_f1},
    }


label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


print(f"Available label_map: {label_map}")

predictions, label_ids, _ = trainer.predict(eval_dataset)


print("Predictions shape:", predictions.shape)
print("Label IDs length:", len(label_ids))

# Get predicted labels by taking argmax across the last axis (for multi-class classification)
predicted_labels = predictions.argmax(axis=-1)

# Ensure label_ids and predicted_labels have the same length
assert len(label_ids) == len(predicted_labels), "Mismatch between label_ids and predicted_labels length!"

# Now, calculate metrics
metrics = calculate_overlap_and_strict_metrics(predicted_labels, label_ids, label_map)

# Print metrics
print("Strict Matching Metrics:")
print(f"Precision: {metrics['strict']['precision']:.4f}, Recall: {metrics['strict']['recall']:.4f}, F1: {metrics['strict']['f1']:.4f}")
print("Overlap Matching Metrics:")
print(f"Precision: {metrics['overlap']['precision']:.4f}, Recall: {metrics['overlap']['recall']:.4f}, F1: {metrics['overlap']['f1']:.4f}")

results_dict = {
    "strict": {
        "precision": metrics["strict"]["precision"],
        "recall": metrics["strict"]["recall"],
        "f1": metrics["strict"]["f1"]
    },
    "overlap": {
        "precision": metrics["overlap"]["precision"],
        "recall": metrics["overlap"]["recall"],
        "f1": metrics["overlap"]["f1"]
    }
}

output_path = '/content/drive/MyDrive/NLP-ProjectFinal/BioBERT/Exp1/Error5e-5/SpanMatching_predictions_results.json'
with open(output_path, 'w') as json_file:
    json.dump(results_dict, json_file, indent=4)

Available label_map: {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


Predictions shape: (2893, 64, 5)
Label IDs length: 2893
Strict Matching Metrics:
Precision: 0.1898, Recall: 0.1521, F1: 0.1688
Overlap Matching Metrics:
Precision: 0.5997, Recall: 0.4849, F1: 0.5362


### **CRF**

In [ ]:
!pip install sklearn_crfsuite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.5 MB/s eta 0:00:00


In [ ]:
import json
import numpy as np
from sklearn_crfsuite import metrics as crf_metrics
from sklearn.metrics import accuracy_score

def compute_crf_metrics(predictions, true_labels, label_map):

    predicted_labels = predictions.argmax(axis=-1)
    predicted_labels = predicted_labels.flatten()
    true_labels = true_labels.flatten()

    mask = true_labels != -100
    predicted_labels = predicted_labels[mask]
    true_labels = true_labels[mask]

    true_labels_tags = [label_map[label] for label in true_labels]
    predicted_labels_tags = [label_map[label] for label in predicted_labels]

    labels = list(label_map.values())[1:]
    sorted_labels = sorted(labels, key=lambda x: (x.split('-')[-1], x.split('-')[0]))
    metrics = crf_metrics.flat_classification_report(
        [true_labels_tags], [predicted_labels_tags],
        labels=sorted_labels, digits=4
    )

    return metrics

label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}

eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


predictions, label_ids, _ = trainer.predict(eval_dataset)


predicted_labels = predictions.argmax(axis=-1)

results_dict = []

for i in range(len(eval_dataset)):
    true_labels = label_ids[i]
    pred_labels = predicted_labels[i]

    # Convert true labels and predicted labels back to BIO tags
    true_labels_str = [label_map.get(label, 'O') for label in true_labels]
    pred_labels_str = [label_map.get(label, 'O') for label in pred_labels]


    results_dict.append({
        'test_id': i,
        'true_labels': true_labels_str,
        'predicted_labels': pred_labels_str,
    })

with open('/content/drive/MyDrive/NLP-ProjectFinal/BioBERT/Exp1/Error5e-5/CRF_results.json', 'w') as json_file:
    json.dump(results_dict, json_file, indent=2)


detailed_metrics = compute_crf_metrics(predictions, label_ids, label_map)


with open('/content/drive/MyDrive/NLP-ProjectFinal/BioBERT/Exp1/Error5e-5/CRF_detailed_metrics_crf.json', 'w') as json_file:
    json.dump(detailed_metrics, json_file, indent=2)

print("\nCRF-style Detailed Metrics:")
print(detailed_metrics)


Evaluation results: {'eval_loss': 0.11069365590810776, 'eval_runtime': 8.9141, 'eval_samples_per_second': 324.54, 'eval_steps_per_second': 5.16, 'epoch': 15.0}

CRF-style Detailed Metrics:
              precision    recall  f1-score   support

       B-CUE     0.3690    0.2518    0.2993       425
       I-CUE     0.0000    0.0000    0.0000        13
     B-SCOPE     0.3324    0.3005    0.3157       416
     I-SCOPE     0.7059    0.6326    0.6673      4690

   micro avg     0.6562    0.5770    0.6141      5544
   macro avg     0.3518    0.2962    0.3206      5544
weighted avg     0.6504    0.5770    0.6111      5544



## Testing with class weights

In [ ]:
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

all_labels = []
for tags in df['Combined_BIO_Tags']:
    all_labels.extend(tags.split())

# Map BIO labels to integers
label_map = {'O': 0, 'B-CUE': 1, 'I-CUE': 2, 'B-SCOPE': 3, 'I-SCOPE': 4}
numeric_labels = [label_map.get(label, -1) for label in all_labels]  # Convert to integers

# Calculate class weights using compute_class_weight
classes = np.unique(numeric_labels)  # Get unique class labels
class_weights = compute_class_weight('balanced', classes=classes, y=numeric_labels)

# Dictionary of class weights
class_weights_dict = {class_id: weight for class_id, weight in zip(classes, class_weights)}

# Output the class weights
print("Class weights:", class_weights_dict)


Class weights: {0: 0.2058965965601889, 1: 91.91340615690169, 2: 2285.3530864197533, 3: 92.97518834756404, 4: 8.256259756478302}


In [ ]:
class_weights_dict = {int(k): v for k, v in class_weights_dict.items()}

with open('/content/drive/MyDrive/NLP-ProjectFinal/BioBERT/Exp2/class_weights_dict.json', 'w') as json_file:
    json.dump(class_weights_dict, json_file, indent=2)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
import numpy as np


all_labels = []
for tags in df['Combined_BIO_Tags']:
    all_labels.extend(tags.split())


label_map = {'O': 0, 'B-CUE': 1, 'I-CUE': 2, 'B-SCOPE': 3, 'I-SCOPE': 4}
numeric_labels = [label_map.get(label, -1) for label in all_labels]

classes = np.unique(numeric_labels)
class_weights = compute_class_weight('balanced', classes=classes, y=numeric_labels)


class_weights_dict = {class_id: weight for class_id, weight in zip(classes, class_weights)}


class_weights_tensor = torch.tensor(list(class_weights_dict.values()), dtype=torch.float)


tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")


def preprocess_data(example):
    encoding = tokenizer(example['Tokenized_Tokens'].split(), padding="max_length", truncation=True, max_length=64, return_tensors="pt",add_special_tokens=False,is_split_into_words=True)
    labels = example['Combined_BIO_Tags'].split()
    encoding['labels'] = [label_map.get(label, 0) for label in labels]
    return encoding

processed_data = []
for index, row in df.iterrows():
    encoding = preprocess_data(row)
    processed_data.append(encoding)


train_dataset = Dataset.from_dict({
    'input_ids': [item['input_ids'].squeeze().tolist() for item in processed_data],
    'attention_mask': [item['attention_mask'].squeeze().tolist() for item in processed_data],
    'labels': [item['labels'] for item in processed_data]
})


train_test_split = train_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

def make_model_contiguous(model):
    for param in model.parameters():
        if not param.is_contiguous():
            param.data = param.data.contiguous()

# Model setup (BioBERT for token classification)
model = AutoModelForTokenClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.1", num_labels=5)  # 5 labels: O, B-CUE, I-CUE, B-SCOPE, I-SCOPE
make_model_contiguous(model)
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define custom loss function with class weights
def custom_loss_fn(output, labels):
    loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor, ignore_index=-100)
    return loss_fct(output.view(-1, model.config.num_labels), labels.view(-1))

class_weights_tensor = torch.tensor(list(class_weights_dict.values()), dtype=torch.float).to(device) # Moved to GPU

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels").to(device)
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor, ignore_index=-100)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    report_to="none",
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=None,

)


for param in model.parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()

trainer.train()


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-21-fc70d57889ed>:110: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


Epoch,Training Loss,Validation Loss
1,1.300000,1.396201
2,1.331700,1.011402
3,0.819200,1.038495
4,0.635100,0.939971
5,0.639600,0.831211
6,0.583300,1.097050
7,0.486200,1.514072
8,0.398300,1.241756
9,0.355700,1.273111
10,0.308100,1.727019


TrainOutput(global_step=2715, training_loss=0.5184294533114846, metrics={'train_runtime': 1910.4019, 'train_samples_per_second': 90.837, 'train_steps_per_second': 1.421, 'total_flos': 5668170317596800.0, 'train_loss': 0.5184294533114846, 'epoch': 15.0})

### Token-level Classification

In [ ]:
import json
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np


def compute_metrics(predictions, true_labels, label_map):

    predicted_labels = predictions.argmax(axis=-1)
    predicted_labels = predicted_labels.flatten()
    true_labels = true_labels.flatten()


    mask = true_labels != -100
    predicted_labels = predicted_labels[mask]
    true_labels = true_labels[mask]

    metrics = {
        'accuracy': accuracy_score(true_labels, predicted_labels),
        'precision_macro': precision_score(true_labels, predicted_labels, average='macro'),
        'recall_macro': recall_score(true_labels, predicted_labels, average='macro'),
        'f1_macro': f1_score(true_labels, predicted_labels, average='macro'),
        'precision_micro': precision_score(true_labels, predicted_labels, average='micro'),
        'recall_micro': recall_score(true_labels, predicted_labels, average='micro'),
        'f1_micro': f1_score(true_labels, predicted_labels, average='micro'),
        'precision_weighted': precision_score(true_labels, predicted_labels, average='weighted'),
        'recall_weighted': recall_score(true_labels, predicted_labels, average='weighted'),
        'f1_weighted': f1_score(true_labels, predicted_labels, average='weighted')
    }


    unique_labels = np.unique(true_labels)
    label_metrics = {}
    for label in unique_labels:
        label_name = label_map.get(label, f'Label_{label}')
        label_metrics[label_name] = {
            'precision': precision_score(true_labels, predicted_labels, labels=[label], average=None)[0],
            'recall': recall_score(true_labels, predicted_labels, labels=[label], average=None)[0],
            'f1': f1_score(true_labels, predicted_labels, labels=[label], average=None)[0]
        }
    metrics['per_label'] = label_metrics

    return metrics


label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

predictions, label_ids, _ = trainer.predict(eval_dataset)

predicted_labels = predictions.argmax(axis=-1)

results_dict = []


for i in range(len(eval_dataset)):
    true_labels = label_ids[i]
    pred_labels = predicted_labels[i]


    true_labels_str = [label_map.get(label, 'O') for label in true_labels]
    pred_labels_str = [label_map.get(label, 'O') for label in pred_labels]


    results_dict.append({
        'test_id': i,
        'true_labels': true_labels_str,
        'predicted_labels': pred_labels_str,
    })


with open('/content/drive/MyDrive/NLP-ProjectFinal/BioBERT/Exp2/Error5e-5/Token_Level_predictions_results.json', 'w') as json_file:
    json.dump(results_dict, json_file, indent=2)

detailed_metrics = compute_metrics(predictions, label_ids, label_map)


with open('/content/drive/MyDrive/NLP-ProjectFinal/BioBERT/Exp2/Error5e-5/Token_Level_detailed_metrics.json', 'w') as json_file:
    json.dump(detailed_metrics, json_file, indent=2)

print("\nOverall Metrics:")
for metric_name, metric_value in detailed_metrics.items():
    if metric_name != 'per_label':
        print(f"{metric_name}: {metric_value:.4f}")

print("\nPer-Label Metrics:")
for label, scores in detailed_metrics['per_label'].items():
    print(f"\nMetrics for {label}:")
    for metric_name, score in scores.items():
        print(f"{metric_name}: {score:.4f}")


Evaluation results: {'eval_loss': 2.458340883255005, 'eval_runtime': 8.8816, 'eval_samples_per_second': 325.729, 'eval_steps_per_second': 5.179, 'epoch': 15.0}

Overall Metrics:
accuracy: 0.9609
precision_macro: 0.3795
recall_macro: 0.5977
f1_macro: 0.4426
precision_micro: 0.9609
recall_micro: 0.9609
f1_micro: 0.9609
precision_weighted: 0.9768
recall_weighted: 0.9609
f1_weighted: 0.9673

Per-Label Metrics:

Metrics for O:
precision: 0.9944
recall: 0.9684
f1: 0.9812

Metrics for B-CUE:
precision: 0.1639
recall: 0.4610
f1: 0.2418

Metrics for I-CUE:
precision: 0.1250
recall: 0.4000
f1: 0.1905

Metrics for B-SCOPE:
precision: 0.1684
recall: 0.3911
f1: 0.2355

Metrics for I-SCOPE:
precision: 0.4458
recall: 0.7682
f1: 0.5642


### **Span Matching Metrics**

In [ ]:

import numpy as np


def extract_spans(labels, label_map):
    spans = []
    start, end, label = None, None, None

    for i, label_id in enumerate(labels):
        if label_id != 0:  # Not 'O' label
            try:
                current_label = label_map[label_id].split('-')[-1]
            except KeyError:
                print(f"KeyError: label_id {label_id} not found in label_map")
                continue
            if start is None:
                start = i
                label = current_label
            elif current_label != label or label_id == 1:
                end = i - 1
                spans.append((start, end, label))
                start, label = i, current_label
        else:
            if start is not None:
                end = i - 1
                spans.append((start, end, label))
                start, label = None, None
    if start is not None:
        spans.append((start, len(labels) - 1, label))

    return spans


def calculate_overlap_and_strict_metrics(predictions, label_ids, label_map):

    strict_tp, strict_fp, strict_fn = 0, 0, 0
    overlap_tp, overlap_fp, overlap_fn = 0, 0, 0
    for i in range(len(predictions)):
        true_labels = label_ids[i]
        pred_labels = predictions[i]


        true_spans = extract_spans(true_labels, label_map)
        pred_spans = extract_spans(pred_labels, label_map)


        for pred_span in pred_spans:
            if pred_span in true_spans:
                strict_tp += 1
            else:
                strict_fp += 1
        for true_span in true_spans:
            if true_span not in pred_spans:
                strict_fn += 1

        for pred_span in pred_spans:
            overlap_found = False
            for true_span in true_spans:
                if (
                    pred_span[2] == true_span[2]  # Same label
                    and (pred_span[0] <= true_span[1] and pred_span[1] >= true_span[0])  # Overlap
                ):
                    overlap_tp += 1
                    overlap_found = True
                    break
            if not overlap_found:
                overlap_fp += 1
        for true_span in true_spans:
            overlap_found = False
            for pred_span in pred_spans:
                if (
                    pred_span[2] == true_span[2]  # Same label
                    and (pred_span[0] <= true_span[1] and pred_span[1] >= true_span[0])  # Overlap
                ):
                    overlap_found = True
                    break
            if not overlap_found:
                overlap_fn += 1

    # Calculate strict metrics
    strict_precision = strict_tp / (strict_tp + strict_fp) if strict_tp + strict_fp > 0 else 0.0
    strict_recall = strict_tp / (strict_tp + strict_fn) if strict_tp + strict_fn > 0 else 0.0
    strict_f1 = 2 * (strict_precision * strict_recall) / (strict_precision + strict_recall) if strict_precision + strict_recall > 0 else 0.0

    # Calculate overlap metrics
    overlap_precision = overlap_tp / (overlap_tp + overlap_fp) if overlap_tp + overlap_fp > 0 else 0.0
    overlap_recall = overlap_tp / (overlap_tp + overlap_fn) if overlap_tp + overlap_fn > 0 else 0.0
    overlap_f1 = 2 * (overlap_precision * overlap_recall) / (overlap_precision + overlap_recall) if overlap_precision + overlap_recall > 0 else 0.0

    return {
        "strict": {"precision": strict_precision, "recall": strict_recall, "f1": strict_f1},
        "overlap": {"precision": overlap_precision, "recall": overlap_recall, "f1": overlap_f1},
    }


label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}  # Ensure the full label_map includes all labels


print(f"Available label_map: {label_map}")


predictions, label_ids, _ = trainer.predict(eval_dataset)


print("Predictions shape:", predictions.shape)
print("Label IDs length:", len(label_ids))


predicted_labels = predictions.argmax(axis=-1)


assert len(label_ids) == len(predicted_labels), "Mismatch between label_ids and predicted_labels length!"


metrics = calculate_overlap_and_strict_metrics(predicted_labels, label_ids, label_map)

print("Strict Matching Metrics:")
print(f"Precision: {metrics['strict']['precision']:.4f}, Recall: {metrics['strict']['recall']:.4f}, F1: {metrics['strict']['f1']:.4f}")
print("Overlap Matching Metrics:")
print(f"Precision: {metrics['overlap']['precision']:.4f}, Recall: {metrics['overlap']['recall']:.4f}, F1: {metrics['overlap']['f1']:.4f}")


results_dict = {
    "strict": {
        "precision": metrics["strict"]["precision"],
        "recall": metrics["strict"]["recall"],
        "f1": metrics["strict"]["f1"]
    },
    "overlap": {
        "precision": metrics["overlap"]["precision"],
        "recall": metrics["overlap"]["recall"],
        "f1": metrics["overlap"]["f1"]
    }
}


output_path = '/content/drive/MyDrive/NLP-ProjectFinal/BioBERT/Exp2/Error5e-5/SM_predictions_results.json'
with open(output_path, 'w') as json_file:
    json.dump(results_dict, json_file, indent=4)

Available label_map: {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


Predictions shape: (2893, 64, 5)
Label IDs length: 2893
Strict Matching Metrics:
Precision: 0.0950, Recall: 0.2429, F1: 0.1366
Overlap Matching Metrics:
Precision: 0.2828, Recall: 0.6839, F1: 0.4001


###**CRF**

In [ ]:
import json
import numpy as np
from sklearn_crfsuite import metrics as crf_metrics
from sklearn.metrics import accuracy_score


def compute_crf_metrics(predictions, true_labels, label_map):

    predicted_labels = predictions.argmax(axis=-1)
    predicted_labels = predicted_labels.flatten()
    true_labels = true_labels.flatten()

    mask = true_labels != -100
    predicted_labels = predicted_labels[mask]
    true_labels = true_labels[mask]


    true_labels_tags = [label_map[label] for label in true_labels]
    predicted_labels_tags = [label_map[label] for label in predicted_labels]


    labels = list(label_map.values())[1:]  # Skip 'O' label for detailed metrics
    sorted_labels = sorted(labels, key=lambda x: (x.split('-')[-1], x.split('-')[0]))
    metrics = crf_metrics.flat_classification_report(
        [true_labels_tags], [predicted_labels_tags],
        labels=sorted_labels, digits=4
    )

    return metrics


label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


predictions, label_ids, _ = trainer.predict(eval_dataset)

predicted_labels = predictions.argmax(axis=-1)

results_dict = []


for i in range(len(eval_dataset)):
    true_labels = label_ids[i]
    pred_labels = predicted_labels[i]


    true_labels_str = [label_map.get(label, 'O') for label in true_labels]
    pred_labels_str = [label_map.get(label, 'O') for label in pred_labels]

    results_dict.append({
        'test_id': i,
        'true_labels': true_labels_str,
        'predicted_labels': pred_labels_str,
    })


with open('/content/drive/MyDrive/NLP-ProjectFinal/BioBERT/Exp2/Error5e-5/CRF_predictions_results.json', 'w') as json_file:
    json.dump(results_dict, json_file, indent=2)


detailed_metrics = compute_crf_metrics(predictions, label_ids, label_map)


with open('/content/drive/MyDrive/NLP-ProjectFinal/BioBERT/Exp2/Error5e-5/CRF_detailed_metrics_crf.json', 'w') as json_file:
    json.dump(detailed_metrics, json_file, indent=2)

print("\nCRF-style Detailed Metrics:")
print(detailed_metrics)


Evaluation results: {'eval_loss': 2.458340883255005, 'eval_runtime': 9.5089, 'eval_samples_per_second': 304.241, 'eval_steps_per_second': 4.838, 'epoch': 15.0}

CRF-style Detailed Metrics:
              precision    recall  f1-score   support

       B-CUE     0.1639    0.4610    0.2418       410
       I-CUE     0.1250    0.4000    0.1905        15
     B-SCOPE     0.1684    0.3911    0.2355       404
     I-SCOPE     0.4458    0.7682    0.5642      4694

   micro avg     0.3871    0.7168    0.5027      5523
   macro avg     0.2258    0.5051    0.3080      5523
weighted avg     0.4037    0.7168    0.5152      5523



# BERT

## Trying new logic Experminet 1

In [ ]:
import pandas as pd


df_abstract = pd.read_csv('/content/drive/.shortcut-targets-by-id/1lhuR_Wtv2FHzNlS9dEdH08FjrmhgsiBy/Dataset/bioscope_abstract.csv')
df_full = pd.read_csv('/content/drive/.shortcut-targets-by-id/1lhuR_Wtv2FHzNlS9dEdH08FjrmhgsiBy/Dataset/bioscope_full.csv')

df = pd.concat([df_abstract, df_full], ignore_index=True)
print(len(df))


14462


In [ ]:
from transformers import AutoTokenizer, BertForSequenceClassification


tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
for idx, row in df.iterrows():

    cue_span = eval(row['cue_span']) if pd.notna(row['cue_span']) and row['cue_span'] != 'nan' else []
    scope_span = eval(row['scope_span']) if pd.notna(row['scope_span']) and row['scope_span'] != 'nan' else []


    tokens = tokenizer(row['sentence'], padding="max_length", truncation=True, max_length=64, return_tensors="pt", return_offsets_mapping=True, add_special_tokens=False)
    offsets = tokens['offset_mapping'][0].tolist()


    cue_bio_tags = ['O'] * len(tokens['input_ids'][0])
    scope_bio_tags = ['O'] * len(tokens['input_ids'][0])
    combined_bio_tags = ['O'] * len(tokens['input_ids'][0])


    cue_start, cue_end = cue_span[0] if cue_span else (None, None)
    scope_start, scope_end = scope_span if scope_span else (None, None)

   ns
    cue_tokens = [
        i for i, (start, end) in enumerate(offsets)
        if start >= cue_start and end <= cue_end and tokens['input_ids'][0][i].item() not in tokenizer.all_special_ids
    ] if cue_start is not None else []

    scope_tokens = [
        i for i, (start, end) in enumerate(offsets)
        if start >= scope_start and end <= scope_end and tokens['input_ids'][0][i].item() not in tokenizer.all_special_ids
    ] if scope_start is not None else []


    modified_scope_tokens = [
        token for token in scope_tokens if not any(start >= cue_start and end <= cue_end for start, end in [offsets[token]])
    ]

    # Assign BIO tags for cue tokens
    for i in cue_tokens:
        if i == cue_tokens[0]:
            cue_bio_tags[i] = 'B-CUE'
        else:
            cue_bio_tags[i] = 'I-CUE'

    # Assign BIO tags for modified scope tokens, ensuring no overlap with cue tags
    for i in modified_scope_tokens:
        if cue_bio_tags[i] == 'O':
            if i == modified_scope_tokens[0]:
                scope_bio_tags[i] = 'B-SCOPE'
            else:
                scope_bio_tags[i] = 'I-SCOPE'

    # Combine cue and scope tags into one set of combined tags
    for i in range(len(cue_bio_tags)):
        if cue_bio_tags[i] != 'O':
            combined_bio_tags[i] = cue_bio_tags[i]
        elif scope_bio_tags[i] != 'O':
            combined_bio_tags[i] = scope_bio_tags[i]

    # Convert tokens and offsets for display in the DataFrame
    tokenized_words = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
    token_spans = [f"{start}-{end}" for start, end in offsets]
    cue_tokens_str = tokenizer.convert_ids_to_tokens([tokens['input_ids'][0][i] for i in cue_tokens])
    scope_tokens_str = tokenizer.convert_ids_to_tokens([tokens['input_ids'][0][i] for i in modified_scope_tokens])

    # Update DataFrame with the calculated values
    df.at[idx, 'Tokenized_Tokens'] = ' '.join(tokenized_words)
    df.at[idx, 'Token_Spans'] = ' '.join(token_spans)
    df.at[idx, 'Cue_BIO_Tags'] = ' '.join(cue_bio_tags)
    df.at[idx, 'Scope_BIO_Tags'] = ' '.join(scope_bio_tags)
    df.at[idx, 'Combined_BIO_Tags'] = ' '.join(combined_bio_tags)
    df.at[idx, 'Cue_Tokens'] = ' '.join(cue_tokens_str)
    df.at[idx, 'Scope_Tokens'] = ' '.join(scope_tokens_str)
    df.at[idx, 'Modified_Cue_Span'] = str(cue_tokens)
    df.at[idx, 'Modified_Scope_Span'] = str(modified_scope_tokens)

print("Done")


Done


In [ ]:
import json

def count_labels_per_negated_row(df, target_column, label_column, output_file):
    negated_rows_counts = []

    for _, row in df.iterrows():
        if row[target_column] == 1:  # Check if the row is negated
            # Count the labels for this row
            label_counts = {}
            labels = row[label_column].split()  # Assuming labels are space-separated
            for label in labels:
                label_counts[label] = label_counts.get(label, 0) + 1

            negated_rows_counts.append({
                "row_index": int(_),  # Convert index to int for JSON compatibility
                "labels": label_counts,
                "sentence": row['sentence'],
                "cue_span": row['cue_span'],
                "cue_tokens": row['Cue_Tokens'],
                "scope_span": row['scope_span'],
                "scope_tokens": row['Scope_Tokens'],
                "modified_cue_span": row['Modified_Cue_Span'],
                "modified_scope_span": row['Modified_Scope_Span']
            })

    # Save to JSON file
    with open(output_file, 'w') as f:
        json.dump(negated_rows_counts, f, indent=4)

    print(f"Results saved to {output_file}")


output_file = "/content/drive/MyDrive/NLP-ProjectFinal/BERT/negated_rows_labels.json"
count_labels_per_negated_row(df, target_column='target', label_column='Combined_BIO_Tags', output_file=output_file)


Results saved to /content/drive/MyDrive/NLP-ProjectFinal/BERT/negated_rows_labels.json


In [ ]:
def count_negated_non_negated(df, target_column):
    counts = {0: 0, 1: 0}

    for _, row in df.iterrows():
        counts[row[target_column]] += 1

    return counts


counts = count_negated_non_negated(df, target_column='target')

print("Counts:")
print(f"Non-negated (0): {counts[0]}")
print(f"Negated (1): {counts[1]}")


Counts:
Non-negated (0): 12368
Negated (1): 2094


In [ ]:
import json
with open('/content/drive/MyDrive/NLP-ProjectFinal/BERT/dffull.json', 'w') as json_file:
    json.dump(df.to_dict(orient='records'), json_file, indent=2)

In [ ]:
# !pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd


tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(example):
    encoding = tokenizer(
        example['Tokenized_Tokens'].split(),  # Convert the space-separated string into a list of tokens
        padding="max_length",
        truncation=True,
        max_length=64,
        return_tensors="pt",
        add_special_tokens=False,
        is_split_into_words=True  # Preserve pre-tokenized structure
    )

    # Aligning BIO labels with the tokens
    labels = example['Combined_BIO_Tags'].split()
    label_map = {'O': 0, 'B-CUE': 1, 'I-CUE': 2, 'B-SCOPE': 3, 'I-SCOPE': 4}
    encoding['labels'] = [label_map.get(label, 0) for label in labels]  # Map BIO tags to integers

    return encoding

processed_data = []
for index, row in df.iterrows():
    encoding = preprocess_data(row)
    processed_data.append(encoding)

train_dataset = Dataset.from_dict({
    'input_ids': [item['input_ids'].squeeze().tolist() for item in processed_data],
    'attention_mask': [item['attention_mask'].squeeze().tolist() for item in processed_data],
    'labels': [item['labels'] for item in processed_data]
})


train_test_split = train_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

def make_model_contiguous(model):
    for param in model.parameters():
        if not param.is_contiguous():
            param.data = param.data.contiguous()

# Model setup (BERT for token classification)
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=5)  # 5 labels: O, B-CUE, I-CUE, B-SCOPE, I-SCOPE
make_model_contiguous(model)

# Check if CUDA (GPU) is available, and if so, move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move model to GPU


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=5e-5,
    eval_strategy="epoch",
    report_to="none",
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)


trainer.train()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-31-26c39ff198b3>:95: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.080300,0.080638
2,0.075500,0.070073
3,0.049800,0.061283
4,0.054400,0.063144
5,0.040300,0.067486
6,0.040900,0.063185
7,0.034100,0.069363
8,0.020100,0.068828
9,0.023300,0.077022
10,0.023600,0.078605


TrainOutput(global_step=2715, training_loss=0.0539133848378891, metrics={'train_runtime': 1902.8552, 'train_samples_per_second': 91.197, 'train_steps_per_second': 1.427, 'total_flos': 5668170317596800.0, 'train_loss': 0.0539133848378891, 'epoch': 15.0})

### Token-level Classification

In [ ]:
import json
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Function to compute metrics, including per-label metrics
def compute_metrics(predictions, true_labels, label_map):
    # Flatten the predictions and labels to be per token
    predicted_labels = predictions.argmax(axis=-1)
    predicted_labels = predicted_labels.flatten()
    true_labels = true_labels.flatten()

    # Mask to ignore padding (where label == -100)
    mask = true_labels != -100
    predicted_labels = predicted_labels[mask]
    true_labels = true_labels[mask]

    # Initialize a dictionary for storing overall metrics
    metrics = {
        'accuracy': accuracy_score(true_labels, predicted_labels),
        'precision_macro': precision_score(true_labels, predicted_labels, average='macro'),
        'recall_macro': recall_score(true_labels, predicted_labels, average='macro'),
        'f1_macro': f1_score(true_labels, predicted_labels, average='macro'),
        'precision_micro': precision_score(true_labels, predicted_labels, average='micro'),
        'recall_micro': recall_score(true_labels, predicted_labels, average='micro'),
        'f1_micro': f1_score(true_labels, predicted_labels, average='micro'),
        'precision_weighted': precision_score(true_labels, predicted_labels, average='weighted'),
        'recall_weighted': recall_score(true_labels, predicted_labels, average='weighted'),
        'f1_weighted': f1_score(true_labels, predicted_labels, average='weighted')
    }

    # Calculate metrics for each label separately
    unique_labels = np.unique(true_labels)
    label_metrics = {}
    for label in unique_labels:
        label_name = label_map.get(label, f'Label_{label}')
        label_metrics[label_name] = {
            'precision': precision_score(true_labels, predicted_labels, labels=[label], average=None)[0],
            'recall': recall_score(true_labels, predicted_labels, labels=[label], average=None)[0],
            'f1': f1_score(true_labels, predicted_labels, labels=[label], average=None)[0]
        }

    # Add per-label metrics to the main metrics dictionary
    metrics['per_label'] = label_metrics

    return metrics


label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


predictions, label_ids, metrics = trainer.predict(eval_dataset)


predicted_labels = predictions.argmax(axis=-1)


reverse_label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


results_dict = []


for i in range(len(eval_dataset)):
    true_labels = label_ids[i]
    pred_labels = predicted_labels[i]


    true_labels_str = [reverse_label_map.get(label, 'O') for label in true_labels]
    pred_labels_str = [reverse_label_map.get(label, 'O') for label in pred_labels]


    results_dict.append({
        'test_id': i,
        'true_labels': true_labels_str,
        'predicted_labels': pred_labels_str,
    })


with open('/content/drive/MyDrive/NLP-ProjectFinal/BERT/Exp1/Error5e-5/Token_level_predictions_results.json', 'w') as json_file:
    json.dump(results_dict, json_file, indent=2)


detailed_metrics = compute_metrics(predictions, label_ids, label_map)


with open('/content/drive/MyDrive/NLP-ProjectFinal/BERT/Exp1/Error5e-5/Token_level_detailed_metrics.json', 'w') as json_file:
    json.dump(detailed_metrics, json_file, indent=2)


print("\nOverall Metrics:")
for metric_name, metric_value in detailed_metrics.items():
    if metric_name != 'per_label':
        print(f"{metric_name}: {metric_value:.4f}")

print("\nPer-Label Metrics:")
for label, scores in detailed_metrics['per_label'].items():
    print(f"\nMetrics for {label}:")
    for metric_name, score in scores.items():
        print(f"{metric_name}: {score:.4f}")


Evaluation results: {'eval_loss': 0.10111614316701889, 'eval_runtime': 8.8395, 'eval_samples_per_second': 327.281, 'eval_steps_per_second': 5.204, 'epoch': 15.0}

Overall Metrics:
accuracy: 0.9811
precision_macro: 0.5345
recall_macro: 0.4692
f1_macro: 0.4933
precision_micro: 0.9811
recall_micro: 0.9811
f1_micro: 0.9811
precision_weighted: 0.9802
recall_weighted: 0.9811
f1_weighted: 0.9806

Per-Label Metrics:

Metrics for O:
precision: 0.9902
recall: 0.9924
f1: 0.9913

Metrics for B-CUE:
precision: 0.3889
recall: 0.3036
f1: 0.3410

Metrics for I-CUE:
precision: 0.2000
recall: 0.0667
f1: 0.1000

Metrics for B-SCOPE:
precision: 0.4090
recall: 0.3350
f1: 0.3683

Metrics for I-SCOPE:
precision: 0.6843
recall: 0.6485
f1: 0.6659


###**Span Macthing Metrics**

In [ ]:

import numpy as np


def extract_spans(labels, label_map):
    spans = []
    start, end, label = None, None, None

    for i, label_id in enumerate(labels):
        if label_id != 0:  # Not 'O' label
            try:
                current_label = label_map[label_id].split('-')[-1]  # Get entity type
            except KeyError:
                print(f"KeyError: label_id {label_id} not found in label_map")
                continue  # Skip if the label_id is not in label_map
            if start is None:  # Start of a new span
                start = i
                label = current_label
            elif current_label != label or label_id == 1:  # New entity or a B- tag
                end = i - 1
                spans.append((start, end, label))
                start, label = i, current_label
        else:
            if start is not None:
                end = i - 1
                spans.append((start, end, label))
                start, label = None, None
    if start is not None:  # End span if it reaches end of sequence
        spans.append((start, len(labels) - 1, label))

    return spans

# Function to calculate both strict and overlap metrics
def calculate_overlap_and_strict_metrics(predictions, label_ids, label_map):
    # Initialize counts for strict and overlap matching
    strict_tp, strict_fp, strict_fn = 0, 0, 0
    overlap_tp, overlap_fp, overlap_fn = 0, 0, 0
    for i in range(len(predictions)):
        true_labels = label_ids[i]
        pred_labels = predictions[i]

        # Extract spans for each method
        true_spans = extract_spans(true_labels, label_map)
        pred_spans = extract_spans(pred_labels, label_map)
        for pred_span in pred_spans:
            if pred_span in true_spans:
                strict_tp += 1
            else:
                strict_fp += 1
        for true_span in true_spans:
            if true_span not in pred_spans:
                strict_fn += 1

        # Overlap matching: Count partial matches as true positives
        for pred_span in pred_spans:
            overlap_found = False
            for true_span in true_spans:
                if (
                    pred_span[2] == true_span[2]  # Same label
                    and (pred_span[0] <= true_span[1] and pred_span[1] >= true_span[0])  # Overlap
                ):
                    overlap_tp += 1
                    overlap_found = True
                    break
            if not overlap_found:
                overlap_fp += 1
        for true_span in true_spans:
            overlap_found = False
            for pred_span in pred_spans:
                if (
                    pred_span[2] == true_span[2]  # Same label
                    and (pred_span[0] <= true_span[1] and pred_span[1] >= true_span[0])  # Overlap
                ):
                    overlap_found = True
                    break
            if not overlap_found:
                overlap_fn += 1

    # Calculate strict metrics
    strict_precision = strict_tp / (strict_tp + strict_fp) if strict_tp + strict_fp > 0 else 0.0
    strict_recall = strict_tp / (strict_tp + strict_fn) if strict_tp + strict_fn > 0 else 0.0
    strict_f1 = 2 * (strict_precision * strict_recall) / (strict_precision + strict_recall) if strict_precision + strict_recall > 0 else 0.0

    # Calculate overlap metrics
    overlap_precision = overlap_tp / (overlap_tp + overlap_fp) if overlap_tp + overlap_fp > 0 else 0.0
    overlap_recall = overlap_tp / (overlap_tp + overlap_fn) if overlap_tp + overlap_fn > 0 else 0.0
    overlap_f1 = 2 * (overlap_precision * overlap_recall) / (overlap_precision + overlap_recall) if overlap_precision + overlap_recall > 0 else 0.0

    return {
        "strict": {"precision": strict_precision, "recall": strict_recall, "f1": strict_f1},
        "overlap": {"precision": overlap_precision, "recall": overlap_recall, "f1": overlap_f1},
    }


label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}  # Ensure the full label_map includes all labels

print(f"Available label_map: {label_map}")


predictions, label_ids, _ = trainer.predict(eval_dataset)


print("Predictions shape:", predictions.shape)
print("Label IDs length:", len(label_ids))


predicted_labels = predictions.argmax(axis=-1)


assert len(label_ids) == len(predicted_labels), "Mismatch between label_ids and predicted_labels length!"


metrics = calculate_overlap_and_strict_metrics(predicted_labels, label_ids, label_map)


print("Strict Matching Metrics:")
print(f"Precision: {metrics['strict']['precision']:.4f}, Recall: {metrics['strict']['recall']:.4f}, F1: {metrics['strict']['f1']:.4f}")
print("Overlap Matching Metrics:")
print(f"Precision: {metrics['overlap']['precision']:.4f}, Recall: {metrics['overlap']['recall']:.4f}, F1: {metrics['overlap']['f1']:.4f}")


results_dict = {
    "strict": {
        "precision": metrics["strict"]["precision"],
        "recall": metrics["strict"]["recall"],
        "f1": metrics["strict"]["f1"]
    },
    "overlap": {
        "precision": metrics["overlap"]["precision"],
        "recall": metrics["overlap"]["recall"],
        "f1": metrics["overlap"]["f1"]
    }
}


output_path = '/content/drive/MyDrive/NLP-ProjectFinal/BERT/Exp1/Error5e-5/SM_predictions_results.json'
with open(output_path, 'w') as json_file:
    json.dump(results_dict, json_file, indent=4)



Available label_map: {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


Predictions shape: (2893, 64, 5)
Label IDs length: 2893
Strict Matching Metrics:
Precision: 0.2240, Recall: 0.1939, F1: 0.2079
Overlap Matching Metrics:
Precision: 0.6159, Recall: 0.5393, F1: 0.5751


### **CRF**

In [ ]:
import json
import numpy as np
from sklearn_crfsuite import metrics as crf_metrics
from sklearn.metrics import accuracy_score


def compute_crf_metrics(predictions, true_labels, label_map):
    # Flatten predictions and labels to be per token
    predicted_labels = predictions.argmax(axis=-1)
    predicted_labels = predicted_labels.flatten()
    true_labels = true_labels.flatten()

    # Mask to ignore padding (where label == -100)
    mask = true_labels != -100
    predicted_labels = predicted_labels[mask]
    true_labels = true_labels[mask]

    # Convert labels back to tag names for CRF metrics
    true_labels_tags = [label_map[label] for label in true_labels]
    predicted_labels_tags = [label_map[label] for label in predicted_labels]

    # Calculate metrics using sklearn_crfsuite metrics
    labels = list(label_map.values())[1:]  # Skip 'O' label for detailed metrics
    sorted_labels = sorted(labels, key=lambda x: (x.split('-')[-1], x.split('-')[0]))  # Sort by tag type for consistency
    metrics = crf_metrics.flat_classification_report(
        [true_labels_tags], [predicted_labels_tags],
        labels=sorted_labels, digits=4
    )

    return metrics


label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


predictions, label_ids, _ = trainer.predict(eval_dataset)

predicted_labels = predictions.argmax(axis=-1)


results_dict = []


for i in range(len(eval_dataset)):
    true_labels = label_ids[i]
    pred_labels = predicted_labels[i]

    true_labels_str = [label_map.get(label, 'O') for label in true_labels]
    pred_labels_str = [label_map.get(label, 'O') for label in pred_labels]


    results_dict.append({
        'test_id': i,
        'true_labels': true_labels_str,
        'predicted_labels': pred_labels_str,
    })


with open('/content/drive/MyDrive/NLP-ProjectFinal/BERT/Exp1/Error5e-5/CRF_predictions_results.json', 'w') as json_file:
    json.dump(results_dict, json_file, indent=2)


detailed_metrics = compute_crf_metrics(predictions, label_ids, label_map)


with open('/content/drive/MyDrive/NLP-ProjectFinal/BERT/Exp1/Error5e-5/CRF_detailed_metrics_crf.json', 'w') as json_file:
    json.dump(detailed_metrics, json_file, indent=2)

print("\nCRF-style Detailed Metrics:")
print(detailed_metrics)


Evaluation results: {'eval_loss': 0.10111614316701889, 'eval_runtime': 9.0259, 'eval_samples_per_second': 320.521, 'eval_steps_per_second': 5.096, 'epoch': 15.0}

CRF-style Detailed Metrics:
              precision    recall  f1-score   support

       B-CUE     0.3889    0.3036    0.3410       415
       I-CUE     0.2000    0.0667    0.1000        15
     B-SCOPE     0.4090    0.3350    0.3683       409
     I-SCOPE     0.6843    0.6485    0.6659      4429

   micro avg     0.6451    0.5953    0.6192      5268
   macro avg     0.4205    0.3384    0.3688      5268
weighted avg     0.6383    0.5953    0.6156      5268



## Testing with class weights

In [ ]:
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


all_labels = []
for tags in df['Combined_BIO_Tags']:
    all_labels.extend(tags.split())


label_map = {'O': 0, 'B-CUE': 1, 'I-CUE': 2, 'B-SCOPE': 3, 'I-SCOPE': 4}
numeric_labels = [label_map.get(label, -1) for label in all_labels]


classes = np.unique(numeric_labels)
class_weights = compute_class_weight('balanced', classes=classes, y=numeric_labels)

class_weights_dict = {class_id: weight for class_id, weight in zip(classes, class_weights)}

print("Class weights:", class_weights_dict)


Class weights: {0: 0.20560981434275782, 1: 91.00963618485743, 2: 2285.3530864197533, 3: 92.32598503740648, 4: 8.759456773766148}


In [ ]:
class_weights_dict = {int(k): v for k, v in class_weights_dict.items()}

with open('/content/drive/MyDrive/NLP-ProjectFinal/BERT/Exp2/class_weights_dict.json', 'w') as json_file:
    json.dump(class_weights_dict, json_file, indent=2)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
import numpy as np


all_labels = []
for tags in df['Combined_BIO_Tags']:
    all_labels.extend(tags.split())


label_map = {'O': 0, 'B-CUE': 1, 'I-CUE': 2, 'B-SCOPE': 3, 'I-SCOPE': 4}
numeric_labels = [label_map.get(label, -1) for label in all_labels]

# Calculate class weights
classes = np.unique(numeric_labels)  # Get unique class labels
class_weights = compute_class_weight('balanced', classes=classes, y=numeric_labels)

# Create a dictionary of class weights
class_weights_dict = {class_id: weight for class_id, weight in zip(classes, class_weights)}

# Convert class weights dictionary to a tensor
class_weights_tensor = torch.tensor(list(class_weights_dict.values()), dtype=torch.float)


tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


def preprocess_data(example):
    encoding = tokenizer(example['Tokenized_Tokens'].split(), padding="max_length", truncation=True, max_length=64, return_tensors="pt",add_special_tokens=False,is_split_into_words=True)
    labels = example['Combined_BIO_Tags'].split()
    encoding['labels'] = [label_map.get(label, 0) for label in labels]
    return encoding

processed_data = []
for index, row in df.iterrows():
    encoding = preprocess_data(row)
    processed_data.append(encoding)

train_dataset = Dataset.from_dict({
    'input_ids': [item['input_ids'].squeeze().tolist() for item in processed_data],
    'attention_mask': [item['attention_mask'].squeeze().tolist() for item in processed_data],
    'labels': [item['labels'] for item in processed_data]
})

train_test_split = train_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

def make_model_contiguous(model):
    for param in model.parameters():
        if not param.is_contiguous():
            param.data = param.data.contiguous()



model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=5)  # 5 labels: O, B-CUE, I-CUE, B-SCOPE, I-SCOPE
make_model_contiguous(model)
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define custom loss function with class weights
def custom_loss_fn(output, labels):
    loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor, ignore_index=-100)
    return loss_fct(output.view(-1, model.config.num_labels), labels.view(-1))

class_weights_tensor = torch.tensor(list(class_weights_dict.values()), dtype=torch.float).to(device) # Moved to GPU

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels").to(device)
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor, ignore_index=-100)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    report_to="none",
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=None,
)

for param in model.parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()

trainer.train()


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-37-494343762907>:111: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


Epoch,Training Loss,Validation Loss
1,1.223300,1.309975
2,1.298800,0.930482
3,0.728400,0.796647
4,0.615800,0.802777
5,0.563500,0.785895
6,0.715300,0.678179
7,0.466200,0.969082
8,0.430000,0.887210
9,0.333000,1.172084
10,0.251700,1.206626


TrainOutput(global_step=2715, training_loss=0.4867968369803595, metrics={'train_runtime': 1932.1348, 'train_samples_per_second': 89.815, 'train_steps_per_second': 1.405, 'total_flos': 5668170317596800.0, 'train_loss': 0.4867968369803595, 'epoch': 15.0})

###**Token-level Classification**

In [ ]:
import json
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

# Define a function to compute metrics including per-label metrics
def compute_metrics(predictions, true_labels, label_map):
    # Flatten predictions and labels to be per token
    predicted_labels = predictions.argmax(axis=-1)
    predicted_labels = predicted_labels.flatten()
    true_labels = true_labels.flatten()

    # Mask to ignore padding (where label == -100)
    mask = true_labels != -100
    predicted_labels = predicted_labels[mask]
    true_labels = true_labels[mask]

    # Initialize a dictionary for storing overall metrics
    metrics = {
        'accuracy': accuracy_score(true_labels, predicted_labels),
        'precision_macro': precision_score(true_labels, predicted_labels, average='macro'),
        'recall_macro': recall_score(true_labels, predicted_labels, average='macro'),
        'f1_macro': f1_score(true_labels, predicted_labels, average='macro'),
        'precision_micro': precision_score(true_labels, predicted_labels, average='micro'),
        'recall_micro': recall_score(true_labels, predicted_labels, average='micro'),
        'f1_micro': f1_score(true_labels, predicted_labels, average='micro'),
        'precision_weighted': precision_score(true_labels, predicted_labels, average='weighted'),
        'recall_weighted': recall_score(true_labels, predicted_labels, average='weighted'),
        'f1_weighted': f1_score(true_labels, predicted_labels, average='weighted')
    }

    # Calculate metrics for each label separately
    unique_labels = np.unique(true_labels)
    label_metrics = {}
    for label in unique_labels:
        label_name = label_map.get(label, f'Label_{label}')
        label_metrics[label_name] = {
            'precision': precision_score(true_labels, predicted_labels, labels=[label], average=None)[0],
            'recall': recall_score(true_labels, predicted_labels, labels=[label], average=None)[0],
            'f1': f1_score(true_labels, predicted_labels, labels=[label], average=None)[0]
        }

    # Add per-label metrics to the main metrics dictionary
    metrics['per_label'] = label_metrics

    return metrics

# Define the label map for interpreting labels
label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


predictions, label_ids, _ = trainer.predict(eval_dataset)


predicted_labels = predictions.argmax(axis=-1)

results_dict = []


for i in range(len(eval_dataset)):
    true_labels = label_ids[i]
    pred_labels = predicted_labels[i]


    true_labels_str = [label_map.get(label, 'O') for label in true_labels]
    pred_labels_str = [label_map.get(label, 'O') for label in pred_labels]


    results_dict.append({
        'test_id': i,
        'true_labels': true_labels_str,
        'predicted_labels': pred_labels_str,
    })


with open('/content/drive/MyDrive/NLP-ProjectFinal/BERT/Exp2/Error5e-5/predictions_results.json', 'w') as json_file:
    json.dump(results_dict, json_file, indent=2)


detailed_metrics = compute_metrics(predictions, label_ids, label_map)


with open('/content/drive/MyDrive/NLP-ProjectFinal/BERT/Exp2/Error5e-5/detailed_metrics.json', 'w') as json_file:
    json.dump(detailed_metrics, json_file, indent=2)


print("\nOverall Metrics:")
for metric_name, metric_value in detailed_metrics.items():
    if metric_name != 'per_label':
        print(f"{metric_name}: {metric_value:.4f}")

print("\nPer-Label Metrics:")
for label, scores in detailed_metrics['per_label'].items():
    print(f"\nMetrics for {label}:")
    for metric_name, score in scores.items():
        print(f"{metric_name}: {score:.4f}")


Evaluation results: {'eval_loss': 1.9126406908035278, 'eval_runtime': 8.8763, 'eval_samples_per_second': 325.924, 'eval_steps_per_second': 5.182, 'epoch': 15.0}

Overall Metrics:
accuracy: 0.9634
precision_macro: 0.4055
recall_macro: 0.6440
f1_macro: 0.4766
precision_micro: 0.9634
recall_micro: 0.9634
f1_micro: 0.9634
precision_weighted: 0.9801
recall_weighted: 0.9634
f1_weighted: 0.9700

Per-Label Metrics:

Metrics for O:
precision: 0.9966
recall: 0.9690
f1: 0.9826

Metrics for B-CUE:
precision: 0.1723
recall: 0.5639
f1: 0.2640

Metrics for I-CUE:
precision: 0.1818
recall: 0.4000
f1: 0.2500

Metrics for B-SCOPE:
precision: 0.2211
recall: 0.4621
f1: 0.2991

Metrics for I-SCOPE:
precision: 0.4557
recall: 0.8252
f1: 0.5871


###**Span Matching Metrics**

In [ ]:
import numpy as np

# Function to extract spans
def extract_spans(labels, label_map):
    spans = []
    start, end, label = None, None, None

    for i, label_id in enumerate(labels):
        if label_id != 0:  # Not 'O' label
            try:
                current_label = label_map[label_id].split('-')[-1]  # Get entity type
            except KeyError:
                print(f"KeyError: label_id {label_id} not found in label_map")
                continue  # Skip if the label_id is not in label_map
            if start is None:  # Start of a new span
                start = i
                label = current_label
            elif current_label != label or label_id == 1:  # New entity or a B- tag
                end = i - 1
                spans.append((start, end, label))
                start, label = i, current_label
        else:
            if start is not None:
                end = i - 1
                spans.append((start, end, label))
                start, label = None, None
    if start is not None:  # End span if it reaches end of sequence
        spans.append((start, len(labels) - 1, label))

    return spans

# Function to calculate both strict and overlap metrics
def calculate_overlap_and_strict_metrics(predictions, label_ids, label_map):
    # Initialize counts for strict and overlap matching
    strict_tp, strict_fp, strict_fn = 0, 0, 0
    overlap_tp, overlap_fp, overlap_fn = 0, 0, 0
    for i in range(len(predictions)):
        true_labels = label_ids[i]
        pred_labels = predictions[i]

        # Extract spans for each method
        true_spans = extract_spans(true_labels, label_map)
        pred_spans = extract_spans(pred_labels, label_map)

        for pred_span in pred_spans:
            if pred_span in true_spans:
                strict_tp += 1
            else:
                strict_fp += 1
        for true_span in true_spans:
            if true_span not in pred_spans:
                strict_fn += 1

        # Overlap matching: Count partial matches as true positives
        for pred_span in pred_spans:
            overlap_found = False
            for true_span in true_spans:
                if (
                    pred_span[2] == true_span[2]  # Same label
                    and (pred_span[0] <= true_span[1] and pred_span[1] >= true_span[0])  # Overlap
                ):
                    overlap_tp += 1
                    overlap_found = True
                    break
            if not overlap_found:
                overlap_fp += 1
        for true_span in true_spans:
            overlap_found = False
            for pred_span in pred_spans:
                if (
                    pred_span[2] == true_span[2]  # Same label
                    and (pred_span[0] <= true_span[1] and pred_span[1] >= true_span[0])  # Overlap
                ):
                    overlap_found = True
                    break
            if not overlap_found:
                overlap_fn += 1

    # Calculate strict metrics
    strict_precision = strict_tp / (strict_tp + strict_fp) if strict_tp + strict_fp > 0 else 0.0
    strict_recall = strict_tp / (strict_tp + strict_fn) if strict_tp + strict_fn > 0 else 0.0
    strict_f1 = 2 * (strict_precision * strict_recall) / (strict_precision + strict_recall) if strict_precision + strict_recall > 0 else 0.0

    # Calculate overlap metrics
    overlap_precision = overlap_tp / (overlap_tp + overlap_fp) if overlap_tp + overlap_fp > 0 else 0.0
    overlap_recall = overlap_tp / (overlap_tp + overlap_fn) if overlap_tp + overlap_fn > 0 else 0.0
    overlap_f1 = 2 * (overlap_precision * overlap_recall) / (overlap_precision + overlap_recall) if overlap_precision + overlap_recall > 0 else 0.0

    return {
        "strict": {"precision": strict_precision, "recall": strict_recall, "f1": strict_f1},
        "overlap": {"precision": overlap_precision, "recall": overlap_recall, "f1": overlap_f1},
    }


label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


print(f"Available label_map: {label_map}")

predictions, label_ids, _ = trainer.predict(eval_dataset)

print("Predictions shape:", predictions.shape)
print("Label IDs length:", len(label_ids))


predicted_labels = predictions.argmax(axis=-1)


assert len(label_ids) == len(predicted_labels), "Mismatch between label_ids and predicted_labels length!"


metrics = calculate_overlap_and_strict_metrics(predicted_labels, label_ids, label_map)


print("Strict Matching Metrics:")
print(f"Precision: {metrics['strict']['precision']:.4f}, Recall: {metrics['strict']['recall']:.4f}, F1: {metrics['strict']['f1']:.4f}")
print("Overlap Matching Metrics:")
print(f"Precision: {metrics['overlap']['precision']:.4f}, Recall: {metrics['overlap']['recall']:.4f}, F1: {metrics['overlap']['f1']:.4f}")


results_dict = {
    "strict": {
        "precision": metrics["strict"]["precision"],
        "recall": metrics["strict"]["recall"],
        "f1": metrics["strict"]["f1"]
    },
    "overlap": {
        "precision": metrics["overlap"]["precision"],
        "recall": metrics["overlap"]["recall"],
        "f1": metrics["overlap"]["f1"]
    }
}

output_path = '/content/drive/MyDrive/NLP-ProjectFinal/BERT/Exp2/Error5e-5/SM_predictions_results.json'
with open(output_path, 'w') as json_file:
    json.dump(results_dict, json_file, indent=4)


Available label_map: {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


Predictions shape: (2893, 64, 5)
Label IDs length: 2893
Strict Matching Metrics:
Precision: 0.1125, Recall: 0.3021, F1: 0.1639
Overlap Matching Metrics:
Precision: 0.2971, Recall: 0.7548, F1: 0.4264


###**CRF**

In [ ]:
import json
import numpy as np
from sklearn_crfsuite import metrics as crf_metrics
from sklearn.metrics import accuracy_score


def compute_crf_metrics(predictions, true_labels, label_map):
    # Flatten predictions and labels to be per token
    predicted_labels = predictions.argmax(axis=-1)
    predicted_labels = predicted_labels.flatten()
    true_labels = true_labels.flatten()

    # Mask to ignore padding (where label == -100)
    mask = true_labels != -100
    predicted_labels = predicted_labels[mask]
    true_labels = true_labels[mask]

    # Convert labels back to tag names for CRF metrics
    true_labels_tags = [label_map[label] for label in true_labels]
    predicted_labels_tags = [label_map[label] for label in predicted_labels]

    # Calculate metrics using sklearn_crfsuite metrics
    labels = list(label_map.values())[1:]  # Skip 'O' label for detailed metrics
    sorted_labels = sorted(labels, key=lambda x: (x.split('-')[-1], x.split('-')[0]))  # Sort by tag type for consistency
    metrics = crf_metrics.flat_classification_report(
        [true_labels_tags], [predicted_labels_tags],
        labels=sorted_labels, digits=4
    )

    return metrics


label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


predictions, label_ids, _ = trainer.predict(eval_dataset)

predicted_labels = predictions.argmax(axis=-1)


results_dict = []

for i in range(len(eval_dataset)):
    true_labels = label_ids[i]
    pred_labels = predicted_labels[i]

    true_labels_str = [label_map.get(label, 'O') for label in true_labels]
    pred_labels_str = [label_map.get(label, 'O') for label in pred_labels]


    results_dict.append({
        'test_id': i,
        'true_labels': true_labels_str,
        'predicted_labels': pred_labels_str,
    })


with open('/content/drive/MyDrive/NLP-ProjectFinal/BERT/Exp2/Error5e-5/CRF_predictions_results.json', 'w') as json_file:
    json.dump(results_dict, json_file, indent=2)


detailed_metrics = compute_crf_metrics(predictions, label_ids, label_map)


with open('/content/drive/MyDrive/NLP-ProjectFinal/BERT/Exp2/Error5e-5/CRF_detailed_metrics_crf.json', 'w') as json_file:
    json.dump(detailed_metrics, json_file, indent=2)

print("\nCRF-style Detailed Metrics:")
print(detailed_metrics)


Evaluation results: {'eval_loss': 1.9126406908035278, 'eval_runtime': 8.9857, 'eval_samples_per_second': 321.956, 'eval_steps_per_second': 5.119, 'epoch': 15.0}

CRF-style Detailed Metrics:
              precision    recall  f1-score   support

       B-CUE     0.1723    0.5639    0.2640       415
       I-CUE     0.1818    0.4000    0.2500        15
     B-SCOPE     0.2211    0.4621    0.2991       409
     I-SCOPE     0.4557    0.8252    0.5871      4429

   micro avg     0.3978    0.7752    0.5258      5268
   macro avg     0.2577    0.5628    0.3500      5268
weighted avg     0.4144    0.7752    0.5384      5268



# Clinical BERT


## Trying new logic Experminet 1

In [ ]:
import pandas as pd

# Read both CSV files
df_abstract = pd.read_csv('/content/drive/.shortcut-targets-by-id/1lhuR_Wtv2FHzNlS9dEdH08FjrmhgsiBy/Dataset/bioscope_abstract.csv')
df_full = pd.read_csv('/content/drive/.shortcut-targets-by-id/1lhuR_Wtv2FHzNlS9dEdH08FjrmhgsiBy/Dataset/bioscope_full.csv')

df = pd.concat([df_abstract, df_full], ignore_index=True)
print(len(df))


14462


In [ ]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [ ]:
for idx, row in df.iterrows():
    # Parse cue and scope spans, handling NaN values
    cue_span = eval(row['cue_span']) if pd.notna(row['cue_span']) and row['cue_span'] != 'nan' else []
    scope_span = eval(row['scope_span']) if pd.notna(row['scope_span']) and row['scope_span'] != 'nan' else []

    # Tokenize the sentence without adding special tokens
    tokens = tokenizer(row['sentence'], padding="max_length", truncation=True, max_length=64, return_tensors="pt", return_offsets_mapping=True, add_special_tokens=False)
    offsets = tokens['offset_mapping'][0].tolist()

    # Initialize BIO tags for cue, scope, and combined tags
    cue_bio_tags = ['O'] * len(tokens['input_ids'][0])
    scope_bio_tags = ['O'] * len(tokens['input_ids'][0])
    combined_bio_tags = ['O'] * len(tokens['input_ids'][0])

    # Define start and end for cue and scope spans
    cue_start, cue_end = cue_span[0] if cue_span else (None, None)
    scope_start, scope_end = scope_span if scope_span else (None, None)

    # Filter tokens that lie within cue and scope spans, excluding special tokens
    cue_tokens = [
        i for i, (start, end) in enumerate(offsets)
        if start >= cue_start and end <= cue_end and tokens['input_ids'][0][i].item() not in tokenizer.all_special_ids
    ] if cue_start is not None else []

    scope_tokens = [
        i for i, (start, end) in enumerate(offsets)
        if start >= scope_start and end <= scope_end and tokens['input_ids'][0][i].item() not in tokenizer.all_special_ids
    ] if scope_start is not None else []

    # Modify scope tokens to exclude any tokens that overlap with cue tokens
    modified_scope_tokens = [
        token for token in scope_tokens if not any(start >= cue_start and end <= cue_end for start, end in [offsets[token]])
    ]

    # Assign BIO tags for cue tokens
    for i in cue_tokens:
        if i == cue_tokens[0]:
            cue_bio_tags[i] = 'B-CUE'
        else:
            cue_bio_tags[i] = 'I-CUE'

    # Assign BIO tags for modified scope tokens, ensuring no overlap with cue tags
    for i in modified_scope_tokens:
        if cue_bio_tags[i] == 'O':
            if i == modified_scope_tokens[0]:
                scope_bio_tags[i] = 'B-SCOPE'
            else:
                scope_bio_tags[i] = 'I-SCOPE'

    # Combine cue and scope tags into one set of combined tags
    for i in range(len(cue_bio_tags)):
        if cue_bio_tags[i] != 'O':
            combined_bio_tags[i] = cue_bio_tags[i]
        elif scope_bio_tags[i] != 'O':
            combined_bio_tags[i] = scope_bio_tags[i]

    # Convert tokens and offsets for display in the DataFrame
    tokenized_words = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
    token_spans = [f"{start}-{end}" for start, end in offsets]
    cue_tokens_str = tokenizer.convert_ids_to_tokens([tokens['input_ids'][0][i] for i in cue_tokens])
    scope_tokens_str = tokenizer.convert_ids_to_tokens([tokens['input_ids'][0][i] for i in modified_scope_tokens])

    # Update DataFrame with the calculated values
    df.at[idx, 'Tokenized_Tokens'] = ' '.join(tokenized_words)
    df.at[idx, 'Token_Spans'] = ' '.join(token_spans)
    df.at[idx, 'Cue_BIO_Tags'] = ' '.join(cue_bio_tags)
    df.at[idx, 'Scope_BIO_Tags'] = ' '.join(scope_bio_tags)
    df.at[idx, 'Combined_BIO_Tags'] = ' '.join(combined_bio_tags)
    df.at[idx, 'Cue_Tokens'] = ' '.join(cue_tokens_str)
    df.at[idx, 'Scope_Tokens'] = ' '.join(scope_tokens_str)
    df.at[idx, 'Modified_Cue_Span'] = str(cue_tokens)
    df.at[idx, 'Modified_Scope_Span'] = str(modified_scope_tokens)

print("Done")


Done


In [ ]:
import json

def count_labels_per_negated_row(df, target_column, label_column, output_file):
    negated_rows_counts = []

    for _, row in df.iterrows():
        if row[target_column] == 1:  # Check if the row is negated
            # Count the labels for this row
            label_counts = {}
            labels = row[label_column].split()
            for label in labels:
                label_counts[label] = label_counts.get(label, 0) + 1

            negated_rows_counts.append({
                "row_index": int(_),
                "labels": label_counts,
                "sentence": row['sentence'],
                "cue_span": row['cue_span'],
                "cue_tokens": row['Cue_Tokens'],
                "scope_span": row['scope_span'],
                "scope_tokens": row['Scope_Tokens'],
                "modified_cue_span": row['Modified_Cue_Span'],
                "modified_scope_span": row['Modified_Scope_Span']
            })


    with open(output_file, 'w') as f:
        json.dump(negated_rows_counts, f, indent=4)

    print(f"Results saved to {output_file}")

output_file = "/content/drive/MyDrive/NLP-ProjectFinal/ClinicalBERT/negated_rows_labels.json"
count_labels_per_negated_row(df, target_column='target', label_column='Combined_BIO_Tags', output_file=output_file)


Results saved to /content/drive/MyDrive/NLP-ProjectFinal/ClinicalBERT/negated_rows_labels.json


In [ ]:
def count_negated_non_negated(df, target_column):
    counts = {0: 0, 1: 0}

    for _, row in df.iterrows():
        counts[row[target_column]] += 1

    return counts

counts = count_negated_non_negated(df, target_column='target')

print("Counts:")
print(f"Non-negated (0): {counts[0]}")
print(f"Negated (1): {counts[1]}")


Counts:
Non-negated (0): 12368
Negated (1): 2094


In [ ]:
import json
with open('/content/drive/MyDrive/NLP-ProjectFinal/ClinicalBERT/fulldf.json', 'w') as json_file:
    json.dump(df.to_dict(orient='records'), json_file, indent=2)

In [ ]:
# !pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

def preprocess_data(example):
    encoding = tokenizer(
        example['Tokenized_Tokens'].split(),
        padding="max_length",
        truncation=True,
        max_length=64,
        return_tensors="pt",
        add_special_tokens=False,
        is_split_into_words=True  #
    )


    labels = example['Combined_BIO_Tags'].split()  # Assuming 'Combined_BIO_Tags' is a space-separated string of labels
    label_map = {'O': 0, 'B-CUE': 1, 'I-CUE': 2, 'B-SCOPE': 3, 'I-SCOPE': 4}  # Map BIO tags to integers
    encoding['labels'] = [label_map.get(label, 0) for label in labels]  # Convert tags to integers

    return encoding

processed_data = []
for index, row in df.iterrows():
    encoding = preprocess_data(row)
    processed_data.append(encoding)


train_dataset = Dataset.from_dict({
    'input_ids': [item['input_ids'].squeeze().tolist() for item in processed_data],
    'attention_mask': [item['attention_mask'].squeeze().tolist() for item in processed_data],
    'labels': [item['labels'] for item in processed_data]
})


train_test_split = train_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

def make_model_contiguous(model):
    for param in model.parameters():
        if not param.is_contiguous():
            param.data = param.data.contiguous()

model = AutoModelForTokenClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=5)  # 5 labels: O, B-CUE, I-CUE, B-SCOPE, I-SCOPE
make_model_contiguous(model)

# Check if CUDA (GPU) is available, and if so, move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move model to GPU

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=5e-5,
    eval_strategy="epoch",
    report_to="none",
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)


trainer.train()


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-47-9909886cf890>:76: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.105500,0.094185
2,0.087100,0.080131
3,0.062300,0.078479
4,0.061100,0.079030
5,0.046100,0.076417
6,0.044700,0.082631
7,0.034100,0.090054
8,0.024700,0.095830
9,0.025700,0.099370
10,0.025800,0.106741


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

TrainOutput(global_step=2715, training_loss=0.054354070626459944, metrics={'train_runtime': 1907.7342, 'train_samples_per_second': 90.964, 'train_steps_per_second': 1.423, 'total_flos': 5668170317596800.0, 'train_loss': 0.054354070626459944, 'epoch': 15.0})

### Token-level Classification

In [ ]:
import json
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


def compute_metrics(predictions, true_labels, label_map):

    predicted_labels = predictions.argmax(axis=-1)
    predicted_labels = predicted_labels.flatten()
    true_labels = true_labels.flatten()


    mask = true_labels != -100
    predicted_labels = predicted_labels[mask]
    true_labels = true_labels[mask]

    metrics = {
        'accuracy': accuracy_score(true_labels, predicted_labels),
        'precision_macro': precision_score(true_labels, predicted_labels, average='macro'),
        'recall_macro': recall_score(true_labels, predicted_labels, average='macro'),
        'f1_macro': f1_score(true_labels, predicted_labels, average='macro'),
        'precision_micro': precision_score(true_labels, predicted_labels, average='micro'),
        'recall_micro': recall_score(true_labels, predicted_labels, average='micro'),
        'f1_micro': f1_score(true_labels, predicted_labels, average='micro'),
        'precision_weighted': precision_score(true_labels, predicted_labels, average='weighted'),
        'recall_weighted': recall_score(true_labels, predicted_labels, average='weighted'),
        'f1_weighted': f1_score(true_labels, predicted_labels, average='weighted')
    }


    unique_labels = np.unique(true_labels)
    label_metrics = {}
    for label in unique_labels:
        label_name = label_map.get(label, f'Label_{label}')
        label_metrics[label_name] = {
            'precision': precision_score(true_labels, predicted_labels, labels=[label], average=None)[0],
            'recall': recall_score(true_labels, predicted_labels, labels=[label], average=None)[0],
            'f1': f1_score(true_labels, predicted_labels, labels=[label], average=None)[0]
        }


    metrics['per_label'] = label_metrics

    return metrics

# Define your label map
label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


predictions, label_ids, metrics = trainer.predict(eval_dataset)

predicted_labels = predictions.argmax(axis=-1)


reverse_label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}

results_dict = []


for i in range(len(eval_dataset)):
    true_labels = label_ids[i]
    pred_labels = predicted_labels[i]


    true_labels_str = [reverse_label_map.get(label, 'O') for label in true_labels]
    pred_labels_str = [reverse_label_map.get(label, 'O') for label in pred_labels]


    results_dict.append({
        'test_id': i,
        'true_labels': true_labels_str,
        'predicted_labels': pred_labels_str,
    })


with open('/content/drive/MyDrive/NLP-ProjectFinal/ClinicalBERT/Exp1/Error5e-5/Token_level_predictions_results.json', 'w') as json_file:
    json.dump(results_dict, json_file, indent=2)


detailed_metrics = compute_metrics(predictions, label_ids, label_map)


with open('/content/drive/MyDrive/NLP-ProjectFinal/ClinicalBERT/Exp1/Error5e-5/Token_level_detailed_metrics.json', 'w') as json_file:
    json.dump(detailed_metrics, json_file, indent=2)


print("\nOverall Metrics:")
for metric_name, metric_value in detailed_metrics.items():
    if metric_name != 'per_label':
        print(f"{metric_name}: {metric_value:.4f}")

print("\nPer-Label Metrics:")
for label, scores in detailed_metrics['per_label'].items():
    print(f"\nMetrics for {label}:")
    for metric_name, score in scores.items():
        print(f"{metric_name}: {score:.4f}")


Evaluation results: {'eval_loss': 0.12963232398033142, 'eval_runtime': 8.8323, 'eval_samples_per_second': 327.548, 'eval_steps_per_second': 5.208, 'epoch': 15.0}

Overall Metrics:
accuracy: 0.9790
precision_macro: 0.4725
recall_macro: 0.4304
f1_macro: 0.4491
precision_micro: 0.9790
recall_micro: 0.9790
f1_micro: 0.9790
precision_weighted: 0.9776
recall_weighted: 0.9790
f1_weighted: 0.9782

Per-Label Metrics:

Metrics for O:
precision: 0.9884
recall: 0.9919
f1: 0.9902

Metrics for B-CUE:
precision: 0.3661
recall: 0.2634
f1: 0.3064

Metrics for I-CUE:
precision: 0.0000
recall: 0.0000
f1: 0.0000

Metrics for B-SCOPE:
precision: 0.3333
recall: 0.2871
f1: 0.3085

Metrics for I-SCOPE:
precision: 0.6748
recall: 0.6093
f1: 0.6404


###**Span Macthing Metrics**

In [ ]:
import numpy as np


def extract_spans(labels, label_map):
    spans = []
    start, end, label = None, None, None

    for i, label_id in enumerate(labels):
        if label_id != 0:  # Not 'O' label
            try:
                current_label = label_map[label_id].split('-')[-1]  #
            except KeyError:
                print(f"KeyError: label_id {label_id} not found in label_map")
                continue
            if start is None:  # Start of a new span
                start = i
                label = current_label
            elif current_label != label or label_id == 1:  # New entity or a B- tag
                end = i - 1
                spans.append((start, end, label))
                start, label = i, current_label
        else:
            if start is not None:
                end = i - 1
                spans.append((start, end, label))
                start, label = None, None
    if start is not None:  # End span if it reaches end of sequence
        spans.append((start, len(labels) - 1, label))

    return spans


def calculate_overlap_and_strict_metrics(predictions, label_ids, label_map):
    # Initialize counts for strict and overlap matching
    strict_tp, strict_fp, strict_fn = 0, 0, 0
    overlap_tp, overlap_fp, overlap_fn = 0, 0, 0
    for i in range(len(predictions)):
        true_labels = label_ids[i]
        pred_labels = predictions[i]

        true_spans = extract_spans(true_labels, label_map)
        pred_spans = extract_spans(pred_labels, label_map)

        for pred_span in pred_spans:
            if pred_span in true_spans:
                strict_tp += 1
            else:
                strict_fp += 1
        for true_span in true_spans:
            if true_span not in pred_spans:
                strict_fn += 1

        # Overlap matching: Count partial matches as true positives
        for pred_span in pred_spans:
            overlap_found = False
            for true_span in true_spans:
                if (
                    pred_span[2] == true_span[2]  # Same label
                    and (pred_span[0] <= true_span[1] and pred_span[1] >= true_span[0])  # Overlap
                ):
                    overlap_tp += 1
                    overlap_found = True
                    break
            if not overlap_found:
                overlap_fp += 1
        for true_span in true_spans:
            overlap_found = False
            for pred_span in pred_spans:
                if (
                    pred_span[2] == true_span[2]  # Same label
                    and (pred_span[0] <= true_span[1] and pred_span[1] >= true_span[0])  # Overlap
                ):
                    overlap_found = True
                    break
            if not overlap_found:
                overlap_fn += 1

    # Calculate strict metrics
    strict_precision = strict_tp / (strict_tp + strict_fp) if strict_tp + strict_fp > 0 else 0.0
    strict_recall = strict_tp / (strict_tp + strict_fn) if strict_tp + strict_fn > 0 else 0.0
    strict_f1 = 2 * (strict_precision * strict_recall) / (strict_precision + strict_recall) if strict_precision + strict_recall > 0 else 0.0

    # Calculate overlap metrics
    overlap_precision = overlap_tp / (overlap_tp + overlap_fp) if overlap_tp + overlap_fp > 0 else 0.0
    overlap_recall = overlap_tp / (overlap_tp + overlap_fn) if overlap_tp + overlap_fn > 0 else 0.0
    overlap_f1 = 2 * (overlap_precision * overlap_recall) / (overlap_precision + overlap_recall) if overlap_precision + overlap_recall > 0 else 0.0

    return {
        "strict": {"precision": strict_precision, "recall": strict_recall, "f1": strict_f1},
        "overlap": {"precision": overlap_precision, "recall": overlap_recall, "f1": overlap_f1},
    }


label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}  # Ensure the full label_map includes all labels


print(f"Available label_map: {label_map}")

predictions, label_ids, _ = trainer.predict(eval_dataset)


print("Predictions shape:", predictions.shape)
print("Label IDs length:", len(label_ids))


predicted_labels = predictions.argmax(axis=-1)


assert len(label_ids) == len(predicted_labels), "Mismatch between label_ids and predicted_labels length!"


metrics = calculate_overlap_and_strict_metrics(predicted_labels, label_ids, label_map)


print("Strict Matching Metrics:")
print(f"Precision: {metrics['strict']['precision']:.4f}, Recall: {metrics['strict']['recall']:.4f}, F1: {metrics['strict']['f1']:.4f}")
print("Overlap Matching Metrics:")
print(f"Precision: {metrics['overlap']['precision']:.4f}, Recall: {metrics['overlap']['recall']:.4f}, F1: {metrics['overlap']['f1']:.4f}")

results_dict = {
    "strict": {
        "precision": metrics["strict"]["precision"],
        "recall": metrics["strict"]["recall"],
        "f1": metrics["strict"]["f1"]
    },
    "overlap": {
        "precision": metrics["overlap"]["precision"],
        "recall": metrics["overlap"]["recall"],
        "f1": metrics["overlap"]["f1"]
    }
}


output_path = '/content/drive/MyDrive/NLP-ProjectFinal/ClinicalBERT/Exp1/Error5e-5/SM_predictions_results.json'
with open(output_path, 'w') as json_file:
    json.dump(results_dict, json_file, indent=4)


Available label_map: {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


Predictions shape: (2893, 64, 5)
Label IDs length: 2893
Strict Matching Metrics:
Precision: 0.1819, Recall: 0.1471, F1: 0.1627
Overlap Matching Metrics:
Precision: 0.5797, Recall: 0.4768, F1: 0.5232


### **CRF**

In [ ]:
import json
import numpy as np
from sklearn_crfsuite import metrics as crf_metrics
from sklearn.metrics import accuracy_score


def compute_crf_metrics(predictions, true_labels, label_map):

    predicted_labels = predictions.argmax(axis=-1)
    predicted_labels = predicted_labels.flatten()
    true_labels = true_labels.flatten()


    mask = true_labels != -100
    predicted_labels = predicted_labels[mask]
    true_labels = true_labels[mask]


    true_labels_tags = [label_map[label] for label in true_labels]
    predicted_labels_tags = [label_map[label] for label in predicted_labels]

    labels = list(label_map.values())[1:]
    sorted_labels = sorted(labels, key=lambda x: (x.split('-')[-1], x.split('-')[0]))
    metrics = crf_metrics.flat_classification_report(
        [true_labels_tags], [predicted_labels_tags],
        labels=sorted_labels, digits=4
    )

    return metrics


label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


predictions, label_ids, _ = trainer.predict(eval_dataset)


predicted_labels = predictions.argmax(axis=-1)


results_dict = []

for i in range(len(eval_dataset)):
    true_labels = label_ids[i]
    pred_labels = predicted_labels[i]


    true_labels_str = [label_map.get(label, 'O') for label in true_labels]
    pred_labels_str = [label_map.get(label, 'O') for label in pred_labels]

    results_dict.append({
        'test_id': i,
        'true_labels': true_labels_str,
        'predicted_labels': pred_labels_str,
    })


with open('/content/drive/MyDrive/NLP-ProjectFinal/ClinicalBERT/Exp1/Error5e-5/CRF_predictions_results.json', 'w') as json_file:
    json.dump(results_dict, json_file, indent=2)


detailed_metrics = compute_crf_metrics(predictions, label_ids, label_map)


with open('/content/drive/MyDrive/NLP-ProjectFinal/ClinicalBERT/Exp1/Error5e-5/CRF_detailed_metrics_crf.json', 'w') as json_file:
    json.dump(detailed_metrics, json_file, indent=2)


print("\nCRF-style Detailed Metrics:")
print(detailed_metrics)


Evaluation results: {'eval_loss': 0.12963232398033142, 'eval_runtime': 9.0319, 'eval_samples_per_second': 320.309, 'eval_steps_per_second': 5.093, 'epoch': 15.0}

CRF-style Detailed Metrics:
              precision    recall  f1-score   support

       B-CUE     0.3661    0.2634    0.3064       410
       I-CUE     0.0000    0.0000    0.0000        15
     B-SCOPE     0.3333    0.2871    0.3085       404
     I-SCOPE     0.6748    0.6093    0.6404      4694

   micro avg     0.6314    0.5584    0.5927      5523
   macro avg     0.3436    0.2900    0.3138      5523
weighted avg     0.6251    0.5584    0.5896      5523



## Testing with class weights

In [ ]:
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


all_labels = []
for tags in df['Combined_BIO_Tags']:
    all_labels.extend(tags.split())

label_map = {'O': 0, 'B-CUE': 1, 'I-CUE': 2, 'B-SCOPE': 3, 'I-SCOPE': 4}
numeric_labels = [label_map.get(label, -1) for label in all_labels]  # Convert to integers


classes = np.unique(numeric_labels)  # Get unique class labels
class_weights = compute_class_weight('balanced', classes=classes, y=numeric_labels)


class_weights_dict = {class_id: weight for class_id, weight in zip(classes, class_weights)}


print("Class weights:", class_weights_dict)


Class weights: {0: 0.2058965965601889, 1: 91.91340615690169, 2: 2285.3530864197533, 3: 92.97518834756404, 4: 8.256259756478302}


In [ ]:
class_weights_dict = {int(k): v for k, v in class_weights_dict.items()}

with open('/content/drive/MyDrive/NLP-ProjectFinal/ClinicalBERT/Exp2/class_weights_dict.json', 'w') as json_file:
    json.dump(class_weights_dict, json_file, indent=2)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
import numpy as np


all_labels = []
for tags in df['Combined_BIO_Tags']:
    all_labels.extend(tags.split())


label_map = {'O': 0, 'B-CUE': 1, 'I-CUE': 2, 'B-SCOPE': 3, 'I-SCOPE': 4}
numeric_labels = [label_map.get(label, -1) for label in all_labels]  # Convert to integers

# Calculate class weights
classes = np.unique(numeric_labels)  # Get unique class labels
class_weights = compute_class_weight('balanced', classes=classes, y=numeric_labels)


class_weights_dict = {class_id: weight for class_id, weight in zip(classes, class_weights)}


class_weights_tensor = torch.tensor(list(class_weights_dict.values()), dtype=torch.float)


tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

def preprocess_data(example):
    encoding = tokenizer(example['Tokenized_Tokens'].split(), padding="max_length", truncation=True, max_length=64, return_tensors="pt",add_special_tokens=False,is_split_into_words=True)
    labels = example['Combined_BIO_Tags'].split()
    encoding['labels'] = [label_map.get(label, 0) for label in labels]
    return encoding

processed_data = []
for index, row in df.iterrows():
    encoding = preprocess_data(row)
    processed_data.append(encoding)


train_dataset = Dataset.from_dict({
    'input_ids': [item['input_ids'].squeeze().tolist() for item in processed_data],
    'attention_mask': [item['attention_mask'].squeeze().tolist() for item in processed_data],
    'labels': [item['labels'] for item in processed_data]
})

train_test_split = train_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

def make_model_contiguous(model):
    for param in model.parameters():
        if not param.is_contiguous():
            param.data = param.data.contiguous()



model = AutoModelForTokenClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=5)  # 5 labels: O, B-CUE, I-CUE, B-SCOPE, I-SCOPE
make_model_contiguous(model)
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define custom loss function with class weights
def custom_loss_fn(output, labels):
    loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor, ignore_index=-100)
    return loss_fct(output.view(-1, model.config.num_labels), labels.view(-1))

class_weights_tensor = torch.tensor(list(class_weights_dict.values()), dtype=torch.float).to(device) # Moved to GPU

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels").to(device)
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor, ignore_index=-100)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    report_to="none",
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=None,
)

for param in model.parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()

trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-53-627695ea69a2>:110: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


Epoch,Training Loss,Validation Loss
1,1.356300,1.359399
2,1.262600,0.977957
3,0.810300,0.930372
4,0.640000,0.901375
5,0.731600,0.855018
6,0.676900,1.081642
7,0.494300,1.322711
8,0.365000,1.490910
9,0.369000,1.498174
10,0.297100,1.743863


TrainOutput(global_step=2715, training_loss=0.5228219780474078, metrics={'train_runtime': 2009.6381, 'train_samples_per_second': 86.351, 'train_steps_per_second': 1.351, 'total_flos': 5668170317596800.0, 'train_loss': 0.5228219780474078, 'epoch': 15.0})

###**Token-level Classification**

In [ ]:
import json
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np


def compute_metrics(predictions, true_labels, label_map):

    predicted_labels = predictions.argmax(axis=-1)
    predicted_labels = predicted_labels.flatten()
    true_labels = true_labels.flatten()


    mask = true_labels != -100
    predicted_labels = predicted_labels[mask]
    true_labels = true_labels[mask]

    # Initialize a dictionary for storing overall metrics
    metrics = {
        'accuracy': accuracy_score(true_labels, predicted_labels),
        'precision_macro': precision_score(true_labels, predicted_labels, average='macro'),
        'recall_macro': recall_score(true_labels, predicted_labels, average='macro'),
        'f1_macro': f1_score(true_labels, predicted_labels, average='macro'),
        'precision_micro': precision_score(true_labels, predicted_labels, average='micro'),
        'recall_micro': recall_score(true_labels, predicted_labels, average='micro'),
        'f1_micro': f1_score(true_labels, predicted_labels, average='micro'),
        'precision_weighted': precision_score(true_labels, predicted_labels, average='weighted'),
        'recall_weighted': recall_score(true_labels, predicted_labels, average='weighted'),
        'f1_weighted': f1_score(true_labels, predicted_labels, average='weighted')
    }

    # Calculate metrics for each label separately
    unique_labels = np.unique(true_labels)
    label_metrics = {}
    for label in unique_labels:
        label_name = label_map.get(label, f'Label_{label}')
        label_metrics[label_name] = {
            'precision': precision_score(true_labels, predicted_labels, labels=[label], average=None)[0],
            'recall': recall_score(true_labels, predicted_labels, labels=[label], average=None)[0],
            'f1': f1_score(true_labels, predicted_labels, labels=[label], average=None)[0]
        }

    # Add per-label metrics to the main metrics dictionary
    metrics['per_label'] = label_metrics

    return metrics


label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

predictions, label_ids, _ = trainer.predict(eval_dataset)
predicted_labels = predictions.argmax(axis=-1)
results_dict = []

for i in range(len(eval_dataset)):
    true_labels = label_ids[i]
    pred_labels = predicted_labels[i]

    # Convert true labels and predicted labels back to BIO tags
    true_labels_str = [label_map.get(label, 'O') for label in true_labels]
    pred_labels_str = [label_map.get(label, 'O') for label in pred_labels]

    # Add to dictionary
    results_dict.append({
        'test_id': i,
        'true_labels': true_labels_str,
        'predicted_labels': pred_labels_str,
    })


with open('/content/drive/MyDrive/NLP-ProjectFinal/ClinicalBERT/Exp2/Error5e-5/Token_level_predictions_results.json', 'w') as json_file:
    json.dump(results_dict, json_file, indent=2)


detailed_metrics = compute_metrics(predictions, label_ids, label_map)


with open('/content/drive/MyDrive/NLP-ProjectFinal/ClinicalBERT/Exp2/Error5e-5/Token_level_detailed_metrics.json', 'w') as json_file:
    json.dump(detailed_metrics, json_file, indent=2)


print("\nOverall Metrics:")
for metric_name, metric_value in detailed_metrics.items():
    if metric_name != 'per_label':
        print(f"{metric_name}: {metric_value:.4f}")

print("\nPer-Label Metrics:")
for label, scores in detailed_metrics['per_label'].items():
    print(f"\nMetrics for {label}:")
    for metric_name, score in scores.items():
        print(f"{metric_name}: {score:.4f}")


Evaluation results: {'eval_loss': 2.5736966133117676, 'eval_runtime': 8.8322, 'eval_samples_per_second': 327.552, 'eval_steps_per_second': 5.208, 'epoch': 15.0}

Overall Metrics:
accuracy: 0.9597
precision_macro: 0.3700
recall_macro: 0.5834
f1_macro: 0.4300
precision_micro: 0.9597
recall_micro: 0.9597
f1_micro: 0.9597
precision_weighted: 0.9758
recall_weighted: 0.9597
f1_weighted: 0.9662

Per-Label Metrics:

Metrics for O:
precision: 0.9937
recall: 0.9676
f1: 0.9805

Metrics for B-CUE:
precision: 0.1514
recall: 0.4073
f1: 0.2208

Metrics for I-CUE:
precision: 0.1017
recall: 0.4000
f1: 0.1622

Metrics for B-SCOPE:
precision: 0.1683
recall: 0.3861
f1: 0.2344

Metrics for I-SCOPE:
precision: 0.4350
recall: 0.7561
f1: 0.5523


###**Span Matching Metrics**

In [ ]:
import numpy as np


def extract_spans(labels, label_map):
    spans = []
    start, end, label = None, None, None

    for i, label_id in enumerate(labels):
        if label_id != 0:  # Not 'O' label
            try:
                current_label = label_map[label_id].split('-')[-1]
            except KeyError:
                print(f"KeyError: label_id {label_id} not found in label_map")
                continue
            if start is None:  # Start of a new span
                start = i
                label = current_label
            elif current_label != label or label_id == 1:  # New entity or a B- tag
                end = i - 1
                spans.append((start, end, label))
                start, label = i, current_label
        else:
            if start is not None:
                end = i - 1
                spans.append((start, end, label))
                start, label = None, None
    if start is not None:  # End span if it reaches end of sequence
        spans.append((start, len(labels) - 1, label))

    return spans

# Function to calculate both strict and overlap metrics
def calculate_overlap_and_strict_metrics(predictions, label_ids, label_map):
    # Initialize counts for strict and overlap matching
    strict_tp, strict_fp, strict_fn = 0, 0, 0
    overlap_tp, overlap_fp, overlap_fn = 0, 0, 0
    for i in range(len(predictions)):
        true_labels = label_ids[i]
        pred_labels = predictions[i]

        # Extract spans for each method
        true_spans = extract_spans(true_labels, label_map)
        pred_spans = extract_spans(pred_labels, label_map)

        for pred_span in pred_spans:
            if pred_span in true_spans:
                strict_tp += 1
            else:
                strict_fp += 1
        for true_span in true_spans:
            if true_span not in pred_spans:
                strict_fn += 1

        # Overlap matching: Count partial matches as true positives
        for pred_span in pred_spans:
            overlap_found = False
            for true_span in true_spans:
                if (
                    pred_span[2] == true_span[2]  # Same label
                    and (pred_span[0] <= true_span[1] and pred_span[1] >= true_span[0])  # Overlap
                ):
                    overlap_tp += 1
                    overlap_found = True
                    break
            if not overlap_found:
                overlap_fp += 1
        for true_span in true_spans:
            overlap_found = False
            for pred_span in pred_spans:
                if (
                    pred_span[2] == true_span[2]  # Same label
                    and (pred_span[0] <= true_span[1] and pred_span[1] >= true_span[0])  # Overlap
                ):
                    overlap_found = True
                    break
            if not overlap_found:
                overlap_fn += 1

    # Calculate strict metrics
    strict_precision = strict_tp / (strict_tp + strict_fp) if strict_tp + strict_fp > 0 else 0.0
    strict_recall = strict_tp / (strict_tp + strict_fn) if strict_tp + strict_fn > 0 else 0.0
    strict_f1 = 2 * (strict_precision * strict_recall) / (strict_precision + strict_recall) if strict_precision + strict_recall > 0 else 0.0

    # Calculate overlap metrics
    overlap_precision = overlap_tp / (overlap_tp + overlap_fp) if overlap_tp + overlap_fp > 0 else 0.0
    overlap_recall = overlap_tp / (overlap_tp + overlap_fn) if overlap_tp + overlap_fn > 0 else 0.0
    overlap_f1 = 2 * (overlap_precision * overlap_recall) / (overlap_precision + overlap_recall) if overlap_precision + overlap_recall > 0 else 0.0

    return {
        "strict": {"precision": strict_precision, "recall": strict_recall, "f1": strict_f1},
        "overlap": {"precision": overlap_precision, "recall": overlap_recall, "f1": overlap_f1},
    }

label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}  # Ensure the full label_map includes all labels


print(f"Available label_map: {label_map}")

predictions, label_ids, _ = trainer.predict(eval_dataset)


print("Predictions shape:", predictions.shape)
print("Label IDs length:", len(label_ids))


predicted_labels = predictions.argmax(axis=-1)


assert len(label_ids) == len(predicted_labels), "Mismatch between label_ids and predicted_labels length!"


metrics = calculate_overlap_and_strict_metrics(predicted_labels, label_ids, label_map)


print("Strict Matching Metrics:")
print(f"Precision: {metrics['strict']['precision']:.4f}, Recall: {metrics['strict']['recall']:.4f}, F1: {metrics['strict']['f1']:.4f}")
print("Overlap Matching Metrics:")
print(f"Precision: {metrics['overlap']['precision']:.4f}, Recall: {metrics['overlap']['recall']:.4f}, F1: {metrics['overlap']['f1']:.4f}")

results_dict = {
    "strict": {
        "precision": metrics["strict"]["precision"],
        "recall": metrics["strict"]["recall"],
        "f1": metrics["strict"]["f1"]
    },
    "overlap": {
        "precision": metrics["overlap"]["precision"],
        "recall": metrics["overlap"]["recall"],
        "f1": metrics["overlap"]["f1"]
    }
}


output_path = '/content/drive/MyDrive/NLP-ProjectFinal/ClinicalBERT/Exp2/Error5e-5/SM_predictions_results.json'
with open(output_path, 'w') as json_file:
    json.dump(results_dict, json_file, indent=4)


Available label_map: {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}


Predictions shape: (2893, 64, 5)
Label IDs length: 2893
Strict Matching Metrics:
Precision: 0.0826, Recall: 0.2064, F1: 0.1180
Overlap Matching Metrics:
Precision: 0.2688, Recall: 0.6430, F1: 0.3791


###**CRF**

In [ ]:
import json
import numpy as np
from sklearn_crfsuite import metrics as crf_metrics
from sklearn.metrics import accuracy_score

# Define a function to compute CRF-style metrics for BIO tags
def compute_crf_metrics(predictions, true_labels, label_map):
    # Flatten predictions and labels to be per token
    predicted_labels = predictions.argmax(axis=-1)
    predicted_labels = predicted_labels.flatten()
    true_labels = true_labels.flatten()

    # Mask to ignore padding (where label == -100)
    mask = true_labels != -100
    predicted_labels = predicted_labels[mask]
    true_labels = true_labels[mask]

    # Convert labels back to tag names for CRF metrics
    true_labels_tags = [label_map[label] for label in true_labels]
    predicted_labels_tags = [label_map[label] for label in predicted_labels]

    # Calculate metrics using sklearn_crfsuite metrics
    labels = list(label_map.values())[1:]  # Skip 'O' label for detailed metrics
    sorted_labels = sorted(labels, key=lambda x: (x.split('-')[-1], x.split('-')[0]))  # Sort by tag type for consistency
    metrics = crf_metrics.flat_classification_report(
        [true_labels_tags], [predicted_labels_tags],
        labels=sorted_labels, digits=4
    )

    return metrics


label_map = {0: 'O', 1: 'B-CUE', 2: 'I-CUE', 3: 'B-SCOPE', 4: 'I-SCOPE'}

eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


predictions, label_ids, _ = trainer.predict(eval_dataset)


predicted_labels = predictions.argmax(axis=-1)


results_dict = []

for i in range(len(eval_dataset)):
    true_labels = label_ids[i]
    pred_labels = predicted_labels[i]


    true_labels_str = [label_map.get(label, 'O') for label in true_labels]
    pred_labels_str = [label_map.get(label, 'O') for label in pred_labels]

    # Add to dictionary
    results_dict.append({
        'test_id': i,
        'true_labels': true_labels_str,
        'predicted_labels': pred_labels_str,
    })


with open('/content/drive/MyDrive/NLP-ProjectFinal/ClinicalBERT/Exp2/Error5e-5/CRF_predictions_results.json', 'w') as json_file:
    json.dump(results_dict, json_file, indent=2)

detailed_metrics = compute_crf_metrics(predictions, label_ids, label_map)


with open('/content/drive/MyDrive/NLP-ProjectFinal/ClinicalBERT/Exp2/Error5e-5/CRF_detailed_metrics_crf.json', 'w') as json_file:
    json.dump(detailed_metrics, json_file, indent=2)


print("\nCRF-style Detailed Metrics:")
print(detailed_metrics)


Evaluation results: {'eval_loss': 2.5736966133117676, 'eval_runtime': 9.0008, 'eval_samples_per_second': 321.416, 'eval_steps_per_second': 5.111, 'epoch': 15.0}

CRF-style Detailed Metrics:
              precision    recall  f1-score   support

       B-CUE     0.1514    0.4073    0.2208       410
       I-CUE     0.1017    0.4000    0.1622        15
     B-SCOPE     0.1683    0.3861    0.2344       404
     I-SCOPE     0.4350    0.7561    0.5523      4694

   micro avg     0.3785    0.7022    0.4918      5523
   macro avg     0.2141    0.4874    0.2924      5523
weighted avg     0.3936    0.7022    0.5034      5523

